In [9]:
import os
os.chdir(r"C:\Users\lizzy\스파르타_파이썬\tmdbapi실습")  # 원하는 경로로 변경
print(os.getcwd())  # 잘 바뀌었는지 확인

C:\Users\lizzy\스파르타_파이썬\tmdbapi실습


In [10]:
import os
from dotenv import load_dotenv

load_dotenv()  # 현재 디렉터리의 .env 파일 읽어서 환경변수로 등록

API_KEY = os.getenv("API_KEY")  # .env에서 API 키 읽어오기

if API_KEY is None:
    raise ValueError("API_KEY가 설정되지 않았습니다. .env 파일에 API_KEY를 확인하세요.")

In [11]:
!pip install aiohttp async-timeout


[notice] A new release of pip is available: 25.2 -> 25.3
[notice] To update, run: python.exe -m pip install --upgrade pip


In [12]:
# ==========================================================
# TMDB TV SERIES + SEASONS FULL ASYNC COLLECTOR
# Series 51 cols / Seasons 15 cols (FINAL + poster_path)
# 2005-01-01 ~ 2015-12-31
# ==========================================================

import os
import asyncio
import aiohttp
import async_timeout
from collections import deque
from datetime import datetime, timedelta
from pathlib import Path
import pandas as pd
from dotenv import load_dotenv

# ==========================================================
# Jupyter async 지원
# ==========================================================
try:
    import nest_asyncio
    nest_asyncio.apply()
    print("✅ nest_asyncio 적용 완료")
except:
    pass

# ==========================================================
# 기본 설정
# ==========================================================
load_dotenv()
API_KEY = os.getenv("API_KEY")
if not API_KEY:
    raise ValueError("TMDB_API_KEY missing")

BASE_URL = "https://api.themoviedb.org/3"
HEADERS = {"accept": "application/json"}
MAX_CALLS_PER_SECOND = 38
TIMEOUT = 10
MAX_RETRIES = 5

START_DATE = "2005-01-01"
END_DATE = "2015-12-31"

# temp
SERIES_TEMP = Path("tv_series_temp.csv")
SEASONS_TEMP = Path("tv_seasons_temp.csv")

# final
SERIES_CSV = "tv_series_2005_2015_FULL.csv"
SEASONS_CSV = "tv_seasons_2005_2015_FULL.csv"
SERIES_PARQ = "tv_series_2005_2015_FULL.parquet"
SEASONS_PARQ = "tv_seasons_2005_2015_FULL.parquet"

# ==========================================================
# Rate limiter
# ==========================================================
class AsyncRateLimiter:
    def __init__(self, max_calls, period=1.0):
        self.max_calls = max_calls
        self.period = period
        self.calls = deque()
        self.lock = asyncio.Lock()

    async def acquire(self):
        async with self.lock:
            now = asyncio.get_event_loop().time()

            # 기간 밖 호출 제거
            while self.calls and now - self.calls[0] > self.period:
                self.calls.popleft()

            # 초당 호출 수 초과 시 sleep
            if len(self.calls) >= self.max_calls:
                sleep_for = self.period - (now - self.calls[0]) + 0.01
                await asyncio.sleep(max(0, sleep_for))

                now = asyncio.get_event_loop().time()
                while self.calls and now - self.calls[0] > self.period:
                    self.calls.popleft()

            self.calls.append(now)

rate_limiter = AsyncRateLimiter(MAX_CALLS_PER_SECOND)

# ==========================================================
# TMDB GET
# ==========================================================
async def tmdb_get(session, path, params=None, retry=0):
    if retry >= MAX_RETRIES:
        return None

    params = params or {}
    params.setdefault("api_key", API_KEY)
    params.setdefault("language", "en-US")

    url = f"{BASE_URL}{path}"

    await rate_limiter.acquire()

    try:
        async with async_timeout.timeout(TIMEOUT):
            async with session.get(url, params=params, headers=HEADERS) as resp:
                # 레이트 리밋
                if resp.status == 429:
                    wait = float(resp.headers.get("Retry-After", 2)) * (2 ** retry)
                    await asyncio.sleep(wait)
                    return await tmdb_get(session, path, params, retry + 1)

                # Not found
                if resp.status == 404:
                    return None

                # 서버 에러
                if 500 <= resp.status < 600:
                    await asyncio.sleep(2 ** retry)
                    return await tmdb_get(session, path, params, retry + 1)

                resp.raise_for_status()
                return await resp.json()

    except:
        await asyncio.sleep(2 ** retry)
        return await tmdb_get(session, path, params, retry + 1)

# ==========================================================
# discover page
# ==========================================================
async def fetch_discover_page(session, page, gte, lte):
    params = {
        "sort_by": "popularity.desc",
        "first_air_date.gte": gte,
        "first_air_date.lte": lte,
        "page": page,
        "include_adult": "true",
    }
    data = await tmdb_get(session, "/discover/tv", params)
    if not data:
        return [], 1, 0
    return data.get("results", []), data.get("total_pages", 1), data.get("total_results", 0)

# ==========================================================
# 분할 수집 (500 page limit 회피)
# ==========================================================
async def collect_ids_in_range(session, start, end, depth=0):
    indent = "  " * depth

    res1, total_pages, total_results = await fetch_discover_page(session, 1, start, end)

    if total_results == 0:
        return set()

    # 500페이지 이하면 그대로 전수 수집
    if total_pages <= 500:
        ids = {r["id"] for r in res1}
        if total_pages > 1:
            tasks = [
                fetch_discover_page(session, p, start, end)
                for p in range(2, total_pages + 1)
            ]
            for coro in asyncio.as_completed(tasks):
                r, _, _ = await coro
                for x in r:
                    ids.add(x["id"])
        return ids

    # 500 페이지 초과 → 날짜 반으로 분할
    start_dt = datetime.strptime(start, "%Y-%m-%d")
    end_dt = datetime.strptime(end, "%Y-%m-%d")
    mid_dt = start_dt + (end_dt - start_dt) / 2
    mid = mid_dt.strftime("%Y-%m-%d")
    right_start = (mid_dt + timedelta(days=1)).strftime("%Y-%m-%d")

    left = await collect_ids_in_range(session, start, mid, depth + 1)
    right = await collect_ids_in_range(session, right_start, end, depth + 1)
    return left | right

# ==========================================================
# helpers
# ==========================================================
def list_to_str(lst, key="name"):
    if not lst:
        return ""
    return ", ".join(str(i.get(key, "")) for i in lst if i.get(key))

def list_ids_to_str(lst, key="id"):
    if not lst:
        return ""
    return ", ".join(str(i.get(key)) for i in lst if i.get(key))

# ==========================================================
# providers
# ==========================================================
async def fetch_providers(session, sid):
    data = await tmdb_get(session, f"/tv/{sid}/watch/providers")
    if not data:
        return ""
    out = set()
    for cc in ["US", "GB", "KR"]:
        if cc in data.get("results", {}):
            p = data["results"][cc].get("flatrate", [])
            for x in p:
                if x.get("provider_name"):
                    out.add(x["provider_name"])
    return ", ".join(sorted(out))

# ==========================================================
# single season (시즌 1개 수집)
# ==========================================================
async def fetch_single_season(session, sid, sname, net_names, net_ids, meta):
    sn = meta.get("season_number")
    if sn is None:
        return None

    data = await tmdb_get(session, f"/tv/{sid}/season/{sn}")

    # 시즌 상세 못 가져오면 meta 기반으로 fallback
    if not data:
        return {
            "_id": f"{sid}_{sn}",
            "season_id": meta.get("id"),
            "series_id": sid,
            "series_name": sname,
            "season_number": sn,
            "name": meta.get("name"),
            "air_date": meta.get("air_date"),
            "overview": meta.get("overview"),
            "vote_average": meta.get("vote_average"),
            "vote_count": meta.get("vote_count"),
            "network_names": net_names,
            "network_ids": net_ids,
            "total_episodes": meta.get("episode_count"),
            "avg_episode_runtime": None,
            "poster_path": meta.get("poster_path"),
        }

    eps = data.get("episodes") or []
    runtimes = [ep.get("runtime") for ep in eps if isinstance(ep.get("runtime"), (int, float))]
    avg_rt = sum(runtimes) / len(runtimes) if runtimes else None

    return {
        "_id": data.get("_id") or f"{sid}_{sn}",
        "season_id": data.get("id"),
        "series_id": sid,
        "series_name": sname,
        "season_number": data.get("season_number"),
        "name": data.get("name"),
        "air_date": data.get("air_date"),
        "overview": data.get("overview"),
        "vote_average": data.get("vote_average"),
        "vote_count": data.get("vote_count"),
        "network_names": net_names,
        "network_ids": net_ids,
        "total_episodes": len(eps),
        "avg_episode_runtime": avg_rt,
        "poster_path": data.get("poster_path"),
    }

# ==========================================================
# TV 상세 + 시즌 (Series 51 cols / Seasons 15 cols)
# ==========================================================
async def fetch_tv_details_and_seasons(session, sid):
    params = {"append_to_response": "reviews,keywords,aggregate_credits"}
    data = await tmdb_get(session, f"/tv/{sid}", params)
    if not data:
        return None, []

    genres = data.get("genres") or []
    networks = data.get("networks") or []
    last_ep = data.get("last_episode_to_air") or {}

    net_names = list_to_str(networks)
    net_ids = list_ids_to_str(networks)

    # ========== SERIES 51개 ==========
    series = {
        "id": data.get("id"),
        "title": data.get("name"),
        "type": "tv_series",
        "adult": data.get("adult"),
        "backdrop_path": data.get("backdrop_path"),
        "created_by": list_to_str(data.get("created_by")),
        "episode_run_time": (
            ", ".join(map(str, data.get("episode_run_time", [])))
            if data.get("episode_run_time") else ""
        ),
        "first_air_date": data.get("first_air_date"),
        "genres": list_to_str(genres),
        "genre_ids": ", ".join(str(g.get("id")) for g in genres if g.get("id")),
        "homepage": data.get("homepage"),
        "in_production": data.get("in_production"),
        "languages": ", ".join(data.get("languages", [])) if data.get("languages") else "",
        "last_air_date": data.get("last_air_date"),

        "last_episode_to_air_id": last_ep.get("id"),
        "last_episode_to_air_name": last_ep.get("name"),
        "last_episode_to_air_overview": last_ep.get("overview"),
        "last_episode_to_air_vote_average": last_ep.get("vote_average"),
        "last_episode_to_air_vote_count": last_ep.get("vote_count"),
        "last_episode_to_air_air_date": last_ep.get("air_date"),
        "last_episode_to_air_episode_number": last_ep.get("episode_number"),
        "last_episode_to_air_production_code": last_ep.get("production_code"),
        "last_episode_to_air_runtime": last_ep.get("runtime"),
        "last_episode_to_air_season_number": last_ep.get("season_number"),
        "last_episode_to_air_show_id": last_ep.get("show_id"),
        "last_episode_to_air_still_path": last_ep.get("still_path"),

        "next_episode_to_air": "" if not data.get("next_episode_to_air") else str(data.get("next_episode_to_air")),
        "networks": net_names,
        "number_of_episodes": data.get("number_of_episodes"),
        "number_of_seasons": data.get("number_of_seasons"),

        "origin_country": ", ".join(data.get("origin_country", [])) if data.get("origin_country") else "",
        "original_language": data.get("original_language"),
        "original_name": data.get("original_name"),
        "overview": data.get("overview"),
        "popularity": data.get("popularity"),
        "poster_path": data.get("poster_path"),

        "production_companies": list_to_str(data.get("production_companies")),
        "production_countries": list_to_str(data.get("production_countries")),

        "seasons": "; ".join(
            f"S{s.get('season_number')}: {s.get('name')} ({s.get('episode_count')} eps)"
            for s in (data.get("seasons") or [])
            if s.get("season_number") is not None
        ),

        "spoken_languages": list_to_str(data.get("spoken_languages")),
        "status": data.get("status"),
        "tagline": data.get("tagline"),
        "type_detail": data.get("type"),
        "vote_average": data.get("vote_average"),
        "vote_count": data.get("vote_count"),
    }

    # 리뷰
    rv_block = data.get("reviews", {})
    rv_items = rv_block.get("results", []) if isinstance(rv_block, dict) else []
    rev = []
    for r in rv_items[:5]:
        author = r.get("author", "")
        rating = r.get("author_details", {}).get("rating")
        rt = f"({rating})" if rating is not None else ""
        cont = (r.get("content") or "").replace("\n", " ").replace("\r", " ")[:200]
        rev.append(f"{author}{rt}: {cont}")
    series["review"] = " || ".join(rev)

    # 키워드
    kw_block = data.get("keywords", {})
    kw_items = kw_block.get("results", []) if isinstance(kw_block, dict) else []
    series["keyword"] = ", ".join([k.get("name") for k in kw_items[:20] if k.get("name")])

    # top cast & crew
    credits = data.get("aggregate_credits", {})
    cast = credits.get("cast") or []
    crew = credits.get("crew") or []

    series["top_cast"] = ", ".join([c.get("name") for c in cast[:4] if c.get("name")])

    dirs = set()
    wrs = set()
    for c in crew:
        nm = c.get("name")
        if not nm:
            continue
        for job in (c.get("jobs") or []):
            jn = job.get("job", "")
            if "Director" in jn:
                dirs.add(nm)
            if jn in ["Writer", "Screenplay", "Story"]:
                wrs.add(nm)

    series["directors"] = ", ".join(sorted(dirs)[:10])
    series["writers"] = ", ".join(sorted(wrs)[:10])

    # providers
    series["providers"] = await fetch_providers(session, sid)

    # 시즌 리스트
    seasons_meta = data.get("seasons") or []
    tasks = [
        fetch_single_season(session, sid, data.get("name") or "", net_names, net_ids, s)
        for s in seasons_meta
        if s.get("season_number") is not None
    ]

    season_records = []
    for coro in asyncio.as_completed(tasks):
        x = await coro
        if x:
            season_records.append(x)

    return series, season_records

# ==========================================================
# 시즌 기반 결측치
# ==========================================================
def fill_series_gaps(series_df, seasons_df):
    print("\n📊 시즌 기반 결측치 보완 시작")

    # episode_run_time
    rtmap = seasons_df.groupby("series_id")["avg_episode_runtime"].mean().round()
    mask = series_df["episode_run_time"].astype(str).str.strip().isin(["", "nan"])
    series_df.loc[mask, "episode_run_time"] = (
        series_df.loc[mask, "id"].map(rtmap).fillna(0).astype(int).astype(str)
    )

    # number_of_episodes
    epmap = seasons_df.groupby("series_id")["total_episodes"].sum()
    mask2 = series_df["number_of_episodes"].isna()
    series_df.loc[mask2, "number_of_episodes"] = (
        series_df.loc[mask2, "id"].map(epmap)
    )

    # number_of_seasons
    seasmap = seasons_df.groupby("series_id")["season_number"].nunique()
    mask3 = series_df["number_of_seasons"].isna()
    series_df.loc[mask3, "number_of_seasons"] = (
        series_df.loc[mask3, "id"].map(seasmap)
    )

    # last_air_date
    lastair = seasons_df.sort_values("air_date").groupby("series_id")["air_date"].last()
    ladmask = series_df["last_air_date"].astype(str).str.strip().isin(["", "nan", "NaT"])
    series_df.loc[ladmask, "last_air_date"] = series_df.loc[ladmask, "id"].map(lastair)

    print("✅ 보완 완료\n")
    return series_df

# ==========================================================
# MAIN
# ==========================================================
async def main():
    print("=" * 90)
    print("🚀 TMDB FULL SERIES + SEASONS COLLECTOR (2005-2015)")
    print("=" * 90)
    t0 = datetime.now()

    # 기존 temp 삭제
    for f in [SERIES_TEMP, SEASONS_TEMP]:
        if f.exists():
            f.unlink()

    async with aiohttp.ClientSession() as session:
        # 1) ID 수집
        print("\n📌 1단계: ID 수집")
        ids = sorted(list(await collect_ids_in_range(session, START_DATE, END_DATE)))
        print(f"✨ 총 ID: {len(ids):,}개")

        # 2) 상세 + 시즌 수집
        print("\n📌 2단계: 상세 + 시즌 수집")
        batch_size = 500
        sem = asyncio.Semaphore(25)

        async def fetch_one(i):
            async with sem:
                return await fetch_tv_details_and_seasons(session, i)

        processed = 0
        for start in range(0, len(ids), batch_size):
            part = ids[start:start+batch_size]
            tasks = [fetch_one(x) for x in part]

            bs = []
            bse = []

            for coro in asyncio.as_completed(tasks):
                s, ss = await coro
                if s:
                    bs.append(s)
                if ss:
                    bse.extend(ss)

                processed += 1
                if processed % 100 == 0:
                    print(f"⏱ {processed:,}/{len(ids):,}")

            # flush
            if bs:
                pd.DataFrame(bs).to_csv(
                    SERIES_TEMP, mode="a", header=not SERIES_TEMP.exists(),
                    index=False, encoding="utf-8-sig"
                )
            if bse:
                pd.DataFrame(bse).to_csv(
                    SEASONS_TEMP, mode="a", header=not SEASONS_TEMP.exists(),
                    index=False, encoding="utf-8-sig"
                )

    # ==========================================================
    # 3) temp → final DF
    # ==========================================================
    df_series = pd.read_csv(SERIES_TEMP)
    df_seasons = pd.read_csv(SEASONS_TEMP) if SEASONS_TEMP.exists() else pd.DataFrame()

    SERIES_COLS = [
        "id","title","type","adult","backdrop_path","created_by","episode_run_time",
        "first_air_date","genres","genre_ids","homepage","in_production","languages",
        "last_air_date","last_episode_to_air_id","last_episode_to_air_name",
        "last_episode_to_air_overview","last_episode_to_air_vote_average",
        "last_episode_to_air_vote_count","last_episode_to_air_air_date",
        "last_episode_to_air_episode_number","last_episode_to_air_production_code",
        "last_episode_to_air_runtime","last_episode_to_air_season_number",
        "last_episode_to_air_show_id","last_episode_to_air_still_path",
        "next_episode_to_air","networks","number_of_episodes","number_of_seasons",
        "origin_country","original_language","original_name","overview","popularity",
        "poster_path",
        "production_companies","production_countries","seasons","spoken_languages",
        "status","tagline","type_detail","vote_average","vote_count","review",
        "keyword","top_cast","directors","writers","providers"
    ]

    SEASON_COLS = [
        "_id","season_id","series_id","series_name","season_number","name","air_date",
        "overview","vote_average","vote_count","network_names","network_ids",
        "total_episodes","avg_episode_runtime","poster_path"
    ]

    df_series = df_series[SERIES_COLS].drop_duplicates(subset=["id"])
    if not df_seasons.empty:
        df_seasons = df_seasons[SEASON_COLS].drop_duplicates(subset=["series_id","season_number"])

    # 4) 시즌 기반 결측치 보완
    if not df_seasons.empty:
        df_series = fill_series_gaps(df_series, df_seasons)

    # 5) 저장
    df_series.to_csv(SERIES_CSV, index=False, encoding="utf-8-sig")
    df_seasons.to_csv(SEASONS_CSV, index=False, encoding="utf-8-sig")

    try:
        df_series.to_parquet(SERIES_PARQ, index=False)
        df_seasons.to_parquet(SEASONS_PARQ, index=False)
    except:
        pass

    for f in [SERIES_TEMP, SEASONS_TEMP]:
        if f.exists():
            f.unlink()

    elapsed = (datetime.now() - t0).total_seconds()/60

    print("="*90)
    print("🎉 수집 완료")
    print("="*90)
    print(f"📌 Series 수: {len(df_series):,}개")
    print(f"📌 Season 수: {len(df_seasons):,}개")
    print(f"📌 Series 컬럼 수: {len(SERIES_COLS)}")
    print(f"📌 Season  컬럼 수: {len(SEASON_COLS)}")
    print(f"⏱ 총 소요시간: {elapsed:.1f}분")
    print("="*90)

# ==========================================================
# RUN
# ==========================================================
if __name__ == "__main__":
    asyncio.run(main())

✅ nest_asyncio 적용 완료
🚀 TMDB FULL SERIES + SEASONS COLLECTOR (2005-2015)

📌 1단계: ID 수집
✨ 총 ID: 42,844개

📌 2단계: 상세 + 시즌 수집
⏱ 100/42,844
⏱ 200/42,844
⏱ 300/42,844
⏱ 400/42,844
⏱ 500/42,844
⏱ 600/42,844
⏱ 700/42,844


KeyboardInterrupt: 

In [ ]:
# ==========================================================
# TMDB TV SERIES + SEASONS FULL ASYNC COLLECTOR (OPTIMIZED)
# Series 51 cols / Seasons 15 cols
# 2005-01-01 ~ 2015-12-31
# ==========================================================

import os
import asyncio
import aiohttp
from collections import deque
from datetime import datetime, timedelta
from pathlib import Path
import pandas as pd
from dotenv import load_dotenv

# ==========================================================
# Jupyter async 지원
# ==========================================================
try:
    import nest_asyncio
    nest_asyncio.apply()
    print("✅ nest_asyncio 적용 완료")
except:
    pass

# ==========================================================
# 기본 설정 (최적화)
# ==========================================================
load_dotenv()
API_KEY = os.getenv("API_KEY")
if not API_KEY:
    raise ValueError("TMDB_API_KEY missing")

BASE_URL = "https://api.themoviedb.org/3"
HEADERS = {"accept": "application/json"}
MAX_CALLS_PER_SECOND = 45  # 38 → 45로 증가
TIMEOUT = 15  # 10 → 15로 증가 (타임아웃 여유)
MAX_RETRIES = 3  # 5 → 3으로 감소 (빠른 실패)

START_DATE = "2005-01-01"
END_DATE = "2015-12-31"

# temp
SERIES_TEMP = Path("tv_series_temp.csv")
SEASONS_TEMP = Path("tv_seasons_temp.csv")

# final
SERIES_CSV = "tv_series_2005_2015_FULL.csv"
SEASONS_CSV = "tv_seasons_2005_2015_FULL.csv"
SERIES_PARQ = "tv_series_2005_2015_FULL.parquet"
SEASONS_PARQ = "tv_seasons_2005_2015_FULL.parquet"

# ==========================================================
# Rate limiter (개선)
# ==========================================================
class AsyncRateLimiter:
    def __init__(self, max_calls, period=1.0):
        self.max_calls = max_calls
        self.period = period
        self.calls = deque()
        self.lock = asyncio.Lock()

    async def acquire(self):
        async with self.lock:
            now = asyncio.get_event_loop().time()
            
            # 기간 밖 호출 제거
            while self.calls and now - self.calls[0] > self.period:
                self.calls.popleft()

            # 초과 시 최소한만 대기
            if len(self.calls) >= self.max_calls:
                sleep_for = self.period - (now - self.calls[0])
                if sleep_for > 0:
                    await asyncio.sleep(sleep_for)
                    now = asyncio.get_event_loop().time()
                    while self.calls and now - self.calls[0] > self.period:
                        self.calls.popleft()

            self.calls.append(now)

rate_limiter = AsyncRateLimiter(MAX_CALLS_PER_SECOND)

# ==========================================================
# TMDB GET (최적화)
# ==========================================================
async def tmdb_get(session, path, params=None, retry=0):
    if retry >= MAX_RETRIES:
        return None

    params = params or {}
    params.setdefault("api_key", API_KEY)
    params.setdefault("language", "en-US")

    url = f"{BASE_URL}{path}"

    await rate_limiter.acquire()

    try:
        async with session.get(url, params=params, headers=HEADERS, timeout=TIMEOUT) as resp:
            # 레이트 리밋
            if resp.status == 429:
                wait = float(resp.headers.get("Retry-After", 1))
                await asyncio.sleep(wait)
                return await tmdb_get(session, path, params, retry + 1)

            # Not found
            if resp.status == 404:
                return None

            # 서버 에러 - 빠른 재시도
            if 500 <= resp.status < 600:
                await asyncio.sleep(0.5 * (retry + 1))
                return await tmdb_get(session, path, params, retry + 1)

            resp.raise_for_status()
            return await resp.json()
    except asyncio.TimeoutError:
        stats["errors"] += 1
        if retry < MAX_RETRIES - 1:
            await asyncio.sleep(1.0 + retry * 0.5) # 대기 시간을 좀 더 확보
            return await tmdb_get(session, path, params, retry + 1)
        # 최종 실패 시
        print(f"❌ [Timeout Fail] {url} after {MAX_RETRIES} retries") # 로그 추가
        return None

# ==========================================================
# discover page
# ==========================================================
async def fetch_discover_page(session, page, gte, lte):
    params = {
        "sort_by": "popularity.desc",
        "first_air_date.gte": gte,
        "first_air_date.lte": lte,
        "page": page,
        "include_adult": "true",
    }
    data = await tmdb_get(session, "/discover/tv", params)
    if not data:
        return [], 1, 0
    return data.get("results", []), data.get("total_pages", 1), data.get("total_results", 0)

# ==========================================================
# 분할 수집 (병렬 처리 최적화)
# ==========================================================
async def collect_ids_in_range(session, start, end, depth=0):
    res1, total_pages, total_results = await fetch_discover_page(session, 1, start, end)

    if total_results == 0:
        return set()

    # 500페이지 이하면 병렬로 전수 수집
    if total_pages <= 500:
        ids = {r["id"] for r in res1}
        if total_pages > 1:
            # 동시에 여러 페이지 수집
            tasks = [
                fetch_discover_page(session, p, start, end)
                for p in range(2, total_pages + 1)
            ]
            results = await asyncio.gather(*tasks, return_exceptions=True)
            for r in results:
                if isinstance(r, tuple):
                    for x in r[0]:
                        ids.add(x["id"])
        return ids

    # 500 페이지 초과 → 날짜 분할 (병렬 처리)
    start_dt = datetime.strptime(start, "%Y-%m-%d")
    end_dt = datetime.strptime(end, "%Y-%m-%d")
    mid_dt = start_dt + (end_dt - start_dt) / 2
    mid = mid_dt.strftime("%Y-%m-%d")
    right_start = (mid_dt + timedelta(days=1)).strftime("%Y-%m-%d")

    # 두 구간을 병렬로 수집
    left, right = await asyncio.gather(
        collect_ids_in_range(session, start, mid, depth + 1),
        collect_ids_in_range(session, right_start, end, depth + 1)
    )
    return left | right

# ==========================================================
# helpers
# ==========================================================
def list_to_str(lst, key="name"):
    if not lst:
        return ""
    return ", ".join(str(i.get(key, "")) for i in lst if i.get(key))

def list_ids_to_str(lst, key="id"):
    if not lst:
        return ""
    return ", ".join(str(i.get(key)) for i in lst if i.get(key))

# ==========================================================
# providers (간소화)
# ==========================================================
async def fetch_providers(session, sid):
    data = await tmdb_get(session, f"/tv/{sid}/watch/providers")
    if not data:
        return ""
    out = set()
    results = data.get("results", {})
    for cc in ["US", "GB", "KR"]:
        if cc in results:
            for x in results[cc].get("flatrate", []):
                if x.get("provider_name"):
                    out.add(x["provider_name"])
    return ", ".join(sorted(out))

# ==========================================================
# single season (최적화)
# ==========================================================
async def fetch_single_season(session, sid, sname, net_names, net_ids, meta):
    sn = meta.get("season_number")
    if sn is None:
        return None

    data = await tmdb_get(session, f"/tv/{sid}/season/{sn}")

    # fallback
    if not data:
        return {
            "_id": f"{sid}_{sn}",
            "season_id": meta.get("id"),
            "series_id": sid,
            "series_name": sname,
            "season_number": sn,
            "name": meta.get("name"),
            "air_date": meta.get("air_date"),
            "overview": meta.get("overview"),
            "vote_average": meta.get("vote_average"),
            "vote_count": meta.get("vote_count"),
            "network_names": net_names,
            "network_ids": net_ids,
            "total_episodes": meta.get("episode_count"),
            "avg_episode_runtime": None,
            "poster_path": meta.get("poster_path"),
        }

    eps = data.get("episodes") or []
    runtimes = [ep.get("runtime") for ep in eps if isinstance(ep.get("runtime"), (int, float))]
    avg_rt = sum(runtimes) / len(runtimes) if runtimes else None

    return {
        "_id": data.get("_id") or f"{sid}_{sn}",
        "season_id": data.get("id"),
        "series_id": sid,
        "series_name": sname,
        "season_number": data.get("season_number"),
        "name": data.get("name"),
        "air_date": data.get("air_date"),
        "overview": data.get("overview"),
        "vote_average": data.get("vote_average"),
        "vote_count": data.get("vote_count"),
        "network_names": net_names,
        "network_ids": net_ids,
        "total_episodes": len(eps),
        "avg_episode_runtime": avg_rt,
        "poster_path": data.get("poster_path"),
    }

# ==========================================================
# TV 상세 + 시즌 (최적화 - gather 사용)
# ==========================================================
async def fetch_tv_details_and_seasons(session, sid):
    params = {"append_to_response": "reviews,keywords,aggregate_credits"}
    data = await tmdb_get(session, f"/tv/{sid}", params)
    if not data:
        return None, []

    genres = data.get("genres") or []
    networks = data.get("networks") or []
    last_ep = data.get("last_episode_to_air") or {}

    net_names = list_to_str(networks)
    net_ids = list_ids_to_str(networks)

    # ========== SERIES 51개 ==========
    series = {
        "id": data.get("id"),
        "title": data.get("name"),
        "type": "tv_series",
        "adult": data.get("adult"),
        "backdrop_path": data.get("backdrop_path"),
        "created_by": list_to_str(data.get("created_by")),
        "episode_run_time": (
            ", ".join(map(str, data.get("episode_run_time", [])))
            if data.get("episode_run_time") else ""
        ),
        "first_air_date": data.get("first_air_date"),
        "genres": list_to_str(genres),
        "genre_ids": ", ".join(str(g.get("id")) for g in genres if g.get("id")),
        "homepage": data.get("homepage"),
        "in_production": data.get("in_production"),
        "languages": ", ".join(data.get("languages", [])) if data.get("languages") else "",
        "last_air_date": data.get("last_air_date"),

        "last_episode_to_air_id": last_ep.get("id"),
        "last_episode_to_air_name": last_ep.get("name"),
        "last_episode_to_air_overview": last_ep.get("overview"),
        "last_episode_to_air_vote_average": last_ep.get("vote_average"),
        "last_episode_to_air_vote_count": last_ep.get("vote_count"),
        "last_episode_to_air_air_date": last_ep.get("air_date"),
        "last_episode_to_air_episode_number": last_ep.get("episode_number"),
        "last_episode_to_air_production_code": last_ep.get("production_code"),
        "last_episode_to_air_runtime": last_ep.get("runtime"),
        "last_episode_to_air_season_number": last_ep.get("season_number"),
        "last_episode_to_air_show_id": last_ep.get("show_id"),
        "last_episode_to_air_still_path": last_ep.get("still_path"),

        "next_episode_to_air": "" if not data.get("next_episode_to_air") else str(data.get("next_episode_to_air")),
        "networks": net_names,
        "number_of_episodes": data.get("number_of_episodes"),
        "number_of_seasons": data.get("number_of_seasons"),

        "origin_country": ", ".join(data.get("origin_country", [])) if data.get("origin_country") else "",
        "original_language": data.get("original_language"),
        "original_name": data.get("original_name"),
        "overview": data.get("overview"),
        "popularity": data.get("popularity"),
        "poster_path": data.get("poster_path"),

        "production_companies": list_to_str(data.get("production_companies")),
        "production_countries": list_to_str(data.get("production_countries")),

        "seasons": "; ".join(
            f"S{s.get('season_number')}: {s.get('name')} ({s.get('episode_count')} eps)"
            for s in (data.get("seasons") or [])
            if s.get("season_number") is not None
        ),

        "spoken_languages": list_to_str(data.get("spoken_languages")),
        "status": data.get("status"),
        "tagline": data.get("tagline"),
        "type_detail": data.get("type"),
        "vote_average": data.get("vote_average"),
        "vote_count": data.get("vote_count"),
    }

    # 리뷰
    rv_block = data.get("reviews", {})
    rv_items = rv_block.get("results", []) if isinstance(rv_block, dict) else []
    rev = []
    for r in rv_items[:5]:
        author = r.get("author", "")
        rating = r.get("author_details", {}).get("rating")
        rt = f"({rating})" if rating is not None else ""
        cont = (r.get("content") or "").replace("\n", " ").replace("\r", " ")[:200]
        rev.append(f"{author}{rt}: {cont}")
    series["review"] = " || ".join(rev)

    # 키워드
    kw_block = data.get("keywords", {})
    kw_items = kw_block.get("results", []) if isinstance(kw_block, dict) else []
    series["keyword"] = ", ".join([k.get("name") for k in kw_items[:20] if k.get("name")])

    # top cast & crew
    credits = data.get("aggregate_credits", {})
    cast = credits.get("cast") or []
    crew = credits.get("crew") or []

    series["top_cast"] = ", ".join([c.get("name") for c in cast[:4] if c.get("name")])

    dirs = set()
    wrs = set()
    for c in crew:
        nm = c.get("name")
        if not nm:
            continue
        for job in (c.get("jobs") or []):
            jn = job.get("job", "")
            if "Director" in jn:
                dirs.add(nm)
            if jn in ["Writer", "Screenplay", "Story"]:
                wrs.add(nm)

    series["directors"] = ", ".join(sorted(dirs)[:10])
    series["writers"] = ", ".join(sorted(wrs)[:10])

    # providers & seasons를 병렬로 수집
    seasons_meta = data.get("seasons") or []
    
    # providers와 모든 시즌을 동시에 수집
    tasks = [fetch_providers(session, sid)]
    tasks.extend([
        fetch_single_season(session, sid, data.get("name") or "", net_names, net_ids, s)
        for s in seasons_meta
        if s.get("season_number") is not None
    ])
    
    results = await asyncio.gather(*tasks, return_exceptions=True)
    
    # 첫 번째는 providers
    series["providers"] = results[0] if isinstance(results[0], str) else ""
    
    # 나머지는 시즌
    season_records = [r for r in results[1:] if r and not isinstance(r, Exception)]

    return series, season_records

# ==========================================================
# 시즌 기반 결측치
# ==========================================================
def fill_series_gaps(series_df, seasons_df):
    print("\n📊 시즌 기반 결측치 보완 시작")

    # episode_run_time
    rtmap = seasons_df.groupby("series_id")["avg_episode_runtime"].mean().round()
    mask = series_df["episode_run_time"].astype(str).str.strip().isin(["", "nan"])
    series_df.loc[mask, "episode_run_time"] = (
        series_df.loc[mask, "id"].map(rtmap).fillna(0).astype(int).astype(str)
    )

    # number_of_episodes
    epmap = seasons_df.groupby("series_id")["total_episodes"].sum()
    mask2 = series_df["number_of_episodes"].isna()
    series_df.loc[mask2, "number_of_episodes"] = (
        series_df.loc[mask2, "id"].map(epmap)
    )

    # number_of_seasons
    seasmap = seasons_df.groupby("series_id")["season_number"].nunique()
    mask3 = series_df["number_of_seasons"].isna()
    series_df.loc[mask3, "number_of_seasons"] = (
        series_df.loc[mask3, "id"].map(seasmap)
    )

    # last_air_date
    lastair = seasons_df.sort_values("air_date").groupby("series_id")["air_date"].last()
    ladmask = series_df["last_air_date"].astype(str).str.strip().isin(["", "nan", "NaT"])
    series_df.loc[ladmask, "last_air_date"] = series_df.loc[ladmask, "id"].map(lastair)

    print("✅ 보완 완료\n")
    return series_df

# ==========================================================
# MAIN (최적화)
# ==========================================================
async def main():
    print("=" * 90)
    print("🚀 TMDB FULL SERIES + SEASONS COLLECTOR (OPTIMIZED)")
    print("=" * 90)
    t0 = datetime.now()

    # 기존 temp 삭제
    for f in [SERIES_TEMP, SEASONS_TEMP]:
        if f.exists():
            f.unlink()

    # TCPConnector 최적화 설정
    connector = aiohttp.TCPConnector(
        limit=100,  # 동시 연결 수 증가
        limit_per_host=50,  # 호스트당 연결 수 증가
        ttl_dns_cache=300  # DNS 캐시
    )
    
    timeout = aiohttp.ClientTimeout(total=30, connect=10, sock_read=15)

    async with aiohttp.ClientSession(connector=connector, timeout=timeout) as session:
        # 1) ID 수집
        print("\n📌 1단계: ID 수집")
        ids = sorted(list(await collect_ids_in_range(session, START_DATE, END_DATE)))
        print(f"✨ 총 ID: {len(ids):,}개")

        # 2) 상세 + 시즌 수집 (최적화)
        print("\n📌 2단계: 상세 + 시즌 수집")
        batch_size = 1000  # 500 → 1000으로 증가
        sem = asyncio.Semaphore(40)  # 25 → 40으로 증가

        async def fetch_one(i):
            async with sem:
                return await fetch_tv_details_and_seasons(session, i)

        processed = 0
        for start in range(0, len(ids), batch_size):
            part = ids[start:start+batch_size]
            tasks = [fetch_one(x) for x in part]

            # gather로 한 번에 수집 (더 빠름)
            results = await asyncio.gather(*tasks, return_exceptions=True)
            
            bs = []
            bse = []
            
            for r in results:
                if isinstance(r, tuple) and r[0]:
                    s, ss = r
                    bs.append(s)
                    if ss:
                        bse.extend(ss)
                
                processed += 1
                if processed % 100 == 0:
                    print(f"⏱ {processed:,}/{len(ids):,}")

            # flush
            if bs:
                pd.DataFrame(bs).to_csv(
                    SERIES_TEMP, mode="a", header=not SERIES_TEMP.exists(),
                    index=False, encoding="utf-8-sig"
                )
            if bse:
                pd.DataFrame(bse).to_csv(
                    SEASONS_TEMP, mode="a", header=not SEASONS_TEMP.exists(),
                    index=False, encoding="utf-8-sig"
                )

    # ==========================================================
    # 3) temp → final DF
    # ==========================================================
    df_series = pd.read_csv(SERIES_TEMP)
    df_seasons = pd.read_csv(SEASONS_TEMP) if SEASONS_TEMP.exists() else pd.DataFrame()

    SERIES_COLS = [
        "id","title","type","adult","backdrop_path","created_by","episode_run_time",
        "first_air_date","genres","genre_ids","homepage","in_production","languages",
        "last_air_date","last_episode_to_air_id","last_episode_to_air_name",
        "last_episode_to_air_overview","last_episode_to_air_vote_average",
        "last_episode_to_air_vote_count","last_episode_to_air_air_date",
        "last_episode_to_air_episode_number","last_episode_to_air_production_code",
        "last_episode_to_air_runtime","last_episode_to_air_season_number",
        "last_episode_to_air_show_id","last_episode_to_air_still_path",
        "next_episode_to_air","networks","number_of_episodes","number_of_seasons",
        "origin_country","original_language","original_name","overview","popularity",
        "poster_path",
        "production_companies","production_countries","seasons","spoken_languages",
        "status","tagline","type_detail","vote_average","vote_count","review",
        "keyword","top_cast","directors","writers","providers"
    ]

    SEASON_COLS = [
        "_id","season_id","series_id","series_name","season_number","name","air_date",
        "overview","vote_average","vote_count","network_names","network_ids",
        "total_episodes","avg_episode_runtime","poster_path"
    ]

    df_series = df_series[SERIES_COLS].drop_duplicates(subset=["id"])
    if not df_seasons.empty:
        df_seasons = df_seasons[SEASON_COLS].drop_duplicates(subset=["series_id","season_number"])

    # 4) 시즌 기반 결측치 보완
    if not df_seasons.empty:
        df_series = fill_series_gaps(df_series, df_seasons)

    # 5) 저장
    df_series.to_csv(SERIES_CSV, index=False, encoding="utf-8-sig")
    df_seasons.to_csv(SEASONS_CSV, index=False, encoding="utf-8-sig")

    try:
        df_series.to_parquet(SERIES_PARQ, index=False)
        df_seasons.to_parquet(SEASONS_PARQ, index=False)
    except:
        pass

    for f in [SERIES_TEMP, SEASONS_TEMP]:
        if f.exists():
            f.unlink()

    elapsed = (datetime.now() - t0).total_seconds()/60

    print("="*90)
    print("🎉 수집 완료")
    print("="*90)
    print(f"📌 Series 수: {len(df_series):,}개")
    print(f"📌 Season 수: {len(df_seasons):,}개")
    print(f"📌 Series 컬럼 수: {len(SERIES_COLS)}")
    print(f"📌 Season  컬럼 수: {len(SEASON_COLS)}")
    print(f"⏱ 총 소요시간: {elapsed:.1f}분")
    print("="*90)

# ==========================================================
# RUN
# ==========================================================
if __name__ == "__main__":
    asyncio.run(main())

✅ nest_asyncio 적용 완료
🚀 TMDB FULL SERIES + SEASONS COLLECTOR (OPTIMIZED)

📌 1단계: ID 수집
✨ 총 ID: 43,869개

📌 2단계: 상세 + 시즌 수집
⏱ 100/43,869
⏱ 200/43,869
⏱ 300/43,869
⏱ 400/43,869
⏱ 500/43,869
⏱ 600/43,869
⏱ 700/43,869
⏱ 800/43,869
⏱ 900/43,869
⏱ 1,000/43,869
⏱ 1,100/43,869
⏱ 1,200/43,869
⏱ 1,300/43,869
⏱ 1,400/43,869
⏱ 1,500/43,869
⏱ 1,600/43,869
⏱ 1,700/43,869
⏱ 1,800/43,869
⏱ 1,900/43,869
⏱ 2,000/43,869
⏱ 2,100/43,869
⏱ 2,200/43,869
⏱ 2,300/43,869
⏱ 2,400/43,869
⏱ 2,500/43,869
⏱ 2,600/43,869
⏱ 2,700/43,869
⏱ 2,800/43,869
⏱ 2,900/43,869
⏱ 3,000/43,869
⏱ 3,100/43,869
⏱ 3,200/43,869
⏱ 3,300/43,869
⏱ 3,400/43,869
⏱ 3,500/43,869
⏱ 3,600/43,869
⏱ 3,700/43,869
⏱ 3,800/43,869
⏱ 3,900/43,869
⏱ 4,000/43,869
⏱ 4,100/43,869
⏱ 4,200/43,869
⏱ 4,300/43,869
⏱ 4,400/43,869
⏱ 4,500/43,869
⏱ 4,600/43,869
⏱ 4,700/43,869
⏱ 4,800/43,869
⏱ 4,900/43,869
⏱ 5,000/43,869


KeyboardInterrupt: 

In [ ]:
# ==========================================================
# TMDB TV SERIES + SEASONS ULTRA-FAST ASYNC COLLECTOR
# Series 51 cols / Seasons 15 cols
# 2005-01-01 ~ 2015-12-31
# ==========================================================

import os
import asyncio
import aiohttp
from collections import deque
from datetime import datetime, timedelta
from pathlib import Path
import pandas as pd
from dotenv import load_dotenv
from concurrent.futures import ThreadPoolExecutor
import time

# ==========================================================
# Jupyter async 지원
# ==========================================================
try:
    import nest_asyncio
    nest_asyncio.apply()
    print("✅ nest_asyncio 적용 완료")
except:
    pass

# ==========================================================
# 기본 설정 (극한 최적화)
# ==========================================================
load_dotenv()
API_KEY = os.getenv("API_KEY")
if not API_KEY:
    raise ValueError("TMDB_API_KEY missing")

BASE_URL = "https://api.themoviedb.org/3"
HEADERS = {"accept": "application/json"}
MAX_CALLS_PER_SECOND = 50  # TMDB 공식 제한에 가깝게
TIMEOUT = aiohttp.ClientTimeout(total=20, connect=5, sock_read=10)
MAX_RETRIES = 2  # 빠른 실패

START_DATE = "2005-01-01"
END_DATE = "2015-12-31"

# temp
SERIES_TEMP = Path("tv_series_temp.csv")
SEASONS_TEMP = Path("tv_seasons_temp.csv")

# final
SERIES_CSV = "tv_series_2005_2015_FULL.csv"
SEASONS_CSV = "tv_seasons_2005_2015_FULL.csv"
SERIES_PARQ = "tv_series_2005_2015_FULL.parquet"
SEASONS_PARQ = "tv_seasons_2005_2015_FULL.parquet"

# 성능 메트릭
stats = {
    "requests": 0,
    "errors": 0,
    "start_time": None
}

# ==========================================================
# Token Bucket Rate Limiter (더 빠름)
# ==========================================================
class TokenBucketLimiter:
    def __init__(self, rate):
        self.rate = rate
        self.tokens = rate
        self.updated_at = time.monotonic()
        self.lock = asyncio.Lock()
    
    async def acquire(self):
        async with self.lock:
            now = time.monotonic()
            # 토큰 보충
            elapsed = now - self.updated_at
            self.tokens = min(self.rate, self.tokens + elapsed * self.rate)
            self.updated_at = now
            
            # 토큰 부족시 대기
            if self.tokens < 1:
                sleep_time = (1 - self.tokens) / self.rate
                await asyncio.sleep(sleep_time)
                self.tokens = 0
            else:
                self.tokens -= 1

rate_limiter = TokenBucketLimiter(MAX_CALLS_PER_SECOND)

# ==========================================================
# TMDB GET (극한 최적화)
# ==========================================================
async def tmdb_get(session, path, params=None, retry=0):
    if retry >= MAX_RETRIES:
        stats["errors"] += 1
        return None

    params = params or {}
    params.setdefault("api_key", API_KEY)
    params.setdefault("language", "en-US")

    url = f"{BASE_URL}{path}"
    
    await rate_limiter.acquire()
    stats["requests"] += 1

    try:
        async with session.get(url, params=params, headers=HEADERS) as resp:
            if resp.status == 429:
                await asyncio.sleep(1)
                return await tmdb_get(session, path, params, retry + 1)
            
            if resp.status == 404:
                return None
            
            if 500 <= resp.status < 600:
                await asyncio.sleep(0.2)
                return await tmdb_get(session, path, params, retry + 1)
            
            resp.raise_for_status()
            return await resp.json()
    
    except asyncio.TimeoutError:
        stats["errors"] += 1
        return None
    except Exception:
        stats["errors"] += 1
        if retry < MAX_RETRIES - 1:
            await asyncio.sleep(0.1)
            return await tmdb_get(session, path, params, retry + 1)
        return None

# ==========================================================
# discover page
# ==========================================================
async def fetch_discover_page(session, page, gte, lte):
    params = {
        "sort_by": "popularity.desc",
        "first_air_date.gte": gte,
        "first_air_date.lte": lte,
        "page": page,
        "include_adult": "true",
    }
    data = await tmdb_get(session, "/discover/tv", params)
    if not data:
        return [], 1, 0
    return data.get("results", []), data.get("total_pages", 1), data.get("total_results", 0)

# ==========================================================
# 분할 수집 (최대 병렬화)
# ==========================================================
async def collect_ids_in_range(session, start, end, depth=0):
    res1, total_pages, total_results = await fetch_discover_page(session, 1, start, end)

    if total_results == 0:
        return set()

    if total_pages <= 500:
        ids = {r["id"] for r in res1}
        if total_pages > 1:
            # 한번에 전부 요청
            results = await asyncio.gather(
                *[fetch_discover_page(session, p, start, end) for p in range(2, total_pages + 1)],
                return_exceptions=True
            )
            for r in results:
                if isinstance(r, tuple):
                    ids.update(x["id"] for x in r[0])
        return ids

    # 날짜 분할
    start_dt = datetime.strptime(start, "%Y-%m-%d")
    end_dt = datetime.strptime(end, "%Y-%m-%d")
    mid_dt = start_dt + (end_dt - start_dt) / 2
    mid = mid_dt.strftime("%Y-%m-%d")
    right_start = (mid_dt + timedelta(days=1)).strftime("%Y-%m-%d")

    left, right = await asyncio.gather(
        collect_ids_in_range(session, start, mid, depth + 1),
        collect_ids_in_range(session, right_start, end, depth + 1)
    )
    return left | right

# ==========================================================
# helpers
# ==========================================================
def list_to_str(lst, key="name"):
    if not lst:
        return ""
    return ", ".join(str(i.get(key, "")) for i in lst if i.get(key))

def list_ids_to_str(lst, key="id"):
    if not lst:
        return ""
    return ", ".join(str(i.get(key)) for i in lst if i.get(key))

# ==========================================================
# providers
# ==========================================================
async def fetch_providers(session, sid):
    data = await tmdb_get(session, f"/tv/{sid}/watch/providers")
    if not data:
        return ""
    out = set()
    results = data.get("results", {})
    for cc in ["US", "GB", "KR"]:
        if cc in results:
            for x in results[cc].get("flatrate", []):
                if x.get("provider_name"):
                    out.add(x["provider_name"])
    return ", ".join(sorted(out))

# ==========================================================
# single season
# ==========================================================
async def fetch_single_season(session, sid, sname, net_names, net_ids, meta):
    sn = meta.get("season_number")
    if sn is None:
        return None

    data = await tmdb_get(session, f"/tv/{sid}/season/{sn}")

    if not data:
        return {
            "_id": f"{sid}_{sn}",
            "season_id": meta.get("id"),
            "series_id": sid,
            "series_name": sname,
            "season_number": sn,
            "name": meta.get("name"),
            "air_date": meta.get("air_date"),
            "overview": meta.get("overview"),
            "vote_average": meta.get("vote_average"),
            "vote_count": meta.get("vote_count"),
            "network_names": net_names,
            "network_ids": net_ids,
            "total_episodes": meta.get("episode_count"),
            "avg_episode_runtime": None,
            "poster_path": meta.get("poster_path"),
        }

    eps = data.get("episodes") or []
    runtimes = [ep.get("runtime") for ep in eps if isinstance(ep.get("runtime"), (int, float))]
    avg_rt = sum(runtimes) / len(runtimes) if runtimes else None

    return {
        "_id": data.get("_id") or f"{sid}_{sn}",
        "season_id": data.get("id"),
        "series_id": sid,
        "series_name": sname,
        "season_number": data.get("season_number"),
        "name": data.get("name"),
        "air_date": data.get("air_date"),
        "overview": data.get("overview"),
        "vote_average": data.get("vote_average"),
        "vote_count": data.get("vote_count"),
        "network_names": net_names,
        "network_ids": net_ids,
        "total_episodes": len(eps),
        "avg_episode_runtime": avg_rt,
        "poster_path": data.get("poster_path"),
    }

# ==========================================================
# TV 상세 + 시즌 (완전 병렬화)
# ==========================================================
async def fetch_tv_details_and_seasons(session, sid):
    params = {"append_to_response": "reviews,keywords,aggregate_credits"}
    data = await tmdb_get(session, f"/tv/{sid}", params)
    if not data:
        return None, []

    genres = data.get("genres") or []
    networks = data.get("networks") or []
    last_ep = data.get("last_episode_to_air") or {}

    net_names = list_to_str(networks)
    net_ids = list_ids_to_str(networks)

    series = {
        "id": data.get("id"),
        "title": data.get("name"),
        "type": "tv_series",
        "adult": data.get("adult"),
        "backdrop_path": data.get("backdrop_path"),
        "created_by": list_to_str(data.get("created_by")),
        "episode_run_time": (
            ", ".join(map(str, data.get("episode_run_time", [])))
            if data.get("episode_run_time") else ""
        ),
        "first_air_date": data.get("first_air_date"),
        "genres": list_to_str(genres),
        "genre_ids": ", ".join(str(g.get("id")) for g in genres if g.get("id")),
        "homepage": data.get("homepage"),
        "in_production": data.get("in_production"),
        "languages": ", ".join(data.get("languages", [])) if data.get("languages") else "",
        "last_air_date": data.get("last_air_date"),

        "last_episode_to_air_id": last_ep.get("id"),
        "last_episode_to_air_name": last_ep.get("name"),
        "last_episode_to_air_overview": last_ep.get("overview"),
        "last_episode_to_air_vote_average": last_ep.get("vote_average"),
        "last_episode_to_air_vote_count": last_ep.get("vote_count"),
        "last_episode_to_air_air_date": last_ep.get("air_date"),
        "last_episode_to_air_episode_number": last_ep.get("episode_number"),
        "last_episode_to_air_production_code": last_ep.get("production_code"),
        "last_episode_to_air_runtime": last_ep.get("runtime"),
        "last_episode_to_air_season_number": last_ep.get("season_number"),
        "last_episode_to_air_show_id": last_ep.get("show_id"),
        "last_episode_to_air_still_path": last_ep.get("still_path"),

        "next_episode_to_air": "" if not data.get("next_episode_to_air") else str(data.get("next_episode_to_air")),
        "networks": net_names,
        "number_of_episodes": data.get("number_of_episodes"),
        "number_of_seasons": data.get("number_of_seasons"),

        "origin_country": ", ".join(data.get("origin_country", [])) if data.get("origin_country") else "",
        "original_language": data.get("original_language"),
        "original_name": data.get("original_name"),
        "overview": data.get("overview"),
        "popularity": data.get("popularity"),
        "poster_path": data.get("poster_path"),

        "production_companies": list_to_str(data.get("production_companies")),
        "production_countries": list_to_str(data.get("production_countries")),

        "seasons": "; ".join(
            f"S{s.get('season_number')}: {s.get('name')} ({s.get('episode_count')} eps)"
            for s in (data.get("seasons") or [])
            if s.get("season_number") is not None
        ),

        "spoken_languages": list_to_str(data.get("spoken_languages")),
        "status": data.get("status"),
        "tagline": data.get("tagline"),
        "type_detail": data.get("type"),
        "vote_average": data.get("vote_average"),
        "vote_count": data.get("vote_count"),
    }

    # 리뷰
    rv_block = data.get("reviews", {})
    rv_items = rv_block.get("results", []) if isinstance(rv_block, dict) else []
    rev = []
    for r in rv_items[:5]:
        author = r.get("author", "")
        rating = r.get("author_details", {}).get("rating")
        rt = f"({rating})" if rating is not None else ""
        cont = (r.get("content") or "").replace("\n", " ").replace("\r", " ")[:200]
        rev.append(f"{author}{rt}: {cont}")
    series["review"] = " || ".join(rev)

    # 키워드
    kw_block = data.get("keywords", {})
    kw_items = kw_block.get("results", []) if isinstance(kw_block, dict) else []
    series["keyword"] = ", ".join([k.get("name") for k in kw_items[:20] if k.get("name")])

    # top cast & crew
    credits = data.get("aggregate_credits", {})
    cast = credits.get("cast") or []
    crew = credits.get("crew") or []

    series["top_cast"] = ", ".join([c.get("name") for c in cast[:4] if c.get("name")])

    dirs = set()
    wrs = set()
    for c in crew:
        nm = c.get("name")
        if not nm:
            continue
        for job in (c.get("jobs") or []):
            jn = job.get("job", "")
            if "Director" in jn:
                dirs.add(nm)
            if jn in ["Writer", "Screenplay", "Story"]:
                wrs.add(nm)

    series["directors"] = ", ".join(sorted(dirs)[:10])
    series["writers"] = ", ".join(sorted(wrs)[:10])

    # providers & seasons 병렬 수집
    seasons_meta = data.get("seasons") or []
    
    tasks = [fetch_providers(session, sid)]
    tasks.extend([
        fetch_single_season(session, sid, data.get("name") or "", net_names, net_ids, s)
        for s in seasons_meta
        if s.get("season_number") is not None
    ])
    
    results = await asyncio.gather(*tasks, return_exceptions=True)
    
    series["providers"] = results[0] if isinstance(results[0], str) else ""
    season_records = [r for r in results[1:] if r and not isinstance(r, Exception)]

    return series, season_records

# ==========================================================
# 비동기 CSV 쓰기 (ThreadPoolExecutor 사용)
# ==========================================================
executor = ThreadPoolExecutor(max_workers=2)

async def save_to_csv_async(df, path, mode, header):
    loop = asyncio.get_event_loop()
    await loop.run_in_executor(
        executor,
        lambda: df.to_csv(path, mode=mode, header=header, index=False, encoding="utf-8-sig")
    )

# ==========================================================
# 시즌 기반 결측치
# ==========================================================
def fill_series_gaps(series_df, seasons_df):
    print("\n📊 시즌 기반 결측치 보완 시작")

    rtmap = seasons_df.groupby("series_id")["avg_episode_runtime"].mean().round()
    mask = series_df["episode_run_time"].astype(str).str.strip().isin(["", "nan"])
    series_df.loc[mask, "episode_run_time"] = (
        series_df.loc[mask, "id"].map(rtmap).fillna(0).astype(int).astype(str)
    )

    epmap = seasons_df.groupby("series_id")["total_episodes"].sum()
    mask2 = series_df["number_of_episodes"].isna()
    series_df.loc[mask2, "number_of_episodes"] = series_df.loc[mask2, "id"].map(epmap)

    seasmap = seasons_df.groupby("series_id")["season_number"].nunique()
    mask3 = series_df["number_of_seasons"].isna()
    series_df.loc[mask3, "number_of_seasons"] = series_df.loc[mask3, "id"].map(seasmap)

    lastair = seasons_df.sort_values("air_date").groupby("series_id")["air_date"].last()
    ladmask = series_df["last_air_date"].astype(str).str.strip().isin(["", "nan", "NaT"])
    series_df.loc[ladmask, "last_air_date"] = series_df.loc[ladmask, "id"].map(lastair)

    print("✅ 보완 완료\n")
    return series_df

# ==========================================================
# MAIN (극한 최적화)
# ==========================================================
async def main():
    print("=" * 90)
    print("🚀 TMDB ULTRA-FAST SERIES + SEASONS COLLECTOR")
    print("=" * 90)
    stats["start_time"] = datetime.now()
    t0 = datetime.now()

    for f in [SERIES_TEMP, SEASONS_TEMP]:
        if f.exists():
            f.unlink()

    # 극한 최적화 설정
    connector = aiohttp.TCPConnector(
        limit=200,  # 100 → 200
        limit_per_host=100,  # 50 → 100
        ttl_dns_cache=600,
        force_close=False,
        enable_cleanup_closed=True
    )

    async with aiohttp.ClientSession(connector=connector, timeout=TIMEOUT) as session:
        # 1) ID 수집
        print("\n📌 1단계: ID 수집")
        ids = sorted(list(await collect_ids_in_range(session, START_DATE, END_DATE)))
        print(f"✨ 총 ID: {len(ids):,}개")

        # 2) 상세 + 시즌 수집
        print("\n📌 2단계: 상세 + 시즌 수집")
        batch_size = 2000  # 1000 → 2000
        sem = asyncio.Semaphore(80)  # 40 → 80

        async def fetch_one(i):
            async with sem:
                return await fetch_tv_details_and_seasons(session, i)

        processed = 0
        for start_idx in range(0, len(ids), batch_size):
            part = ids[start_idx:start_idx+batch_size]
            
            # 전부 병렬로 수집
            results = await asyncio.gather(
                *[fetch_one(x) for x in part],
                return_exceptions=True
            )
            
            bs = []
            bse = []
            
            for r in results:
                if isinstance(r, tuple) and r[0]:
                    s, ss = r
                    bs.append(s)
                    if ss:
                        bse.extend(ss)
                
                processed += 1
                if processed % 200 == 0:
                    elapsed = (datetime.now() - t0).total_seconds()
                    rate = processed / elapsed if elapsed > 0 else 0
                    eta = (len(ids) - processed) / rate / 60 if rate > 0 else 0
                    print(f"⏱ {processed:,}/{len(ids):,} | {rate:.1f} series/s | ETA: {eta:.1f}분 | 요청: {stats['requests']:,} | 에러: {stats['errors']}")

            # 비동기 CSV 쓰기
            if bs:
                await save_to_csv_async(
                    pd.DataFrame(bs),
                    SERIES_TEMP,
                    "a",
                    not SERIES_TEMP.exists()
                )
            if bse:
                await save_to_csv_async(
                    pd.DataFrame(bse),
                    SEASONS_TEMP,
                    "a",
                    not SEASONS_TEMP.exists()
                )

    # 3) 최종 처리
    df_series = pd.read_csv(SERIES_TEMP)
    df_seasons = pd.read_csv(SEASONS_TEMP) if SEASONS_TEMP.exists() else pd.DataFrame()

    SERIES_COLS = [
        "id","title","type","adult","backdrop_path","created_by","episode_run_time",
        "first_air_date","genres","genre_ids","homepage","in_production","languages",
        "last_air_date","last_episode_to_air_id","last_episode_to_air_name",
        "last_episode_to_air_overview","last_episode_to_air_vote_average",
        "last_episode_to_air_vote_count","last_episode_to_air_air_date",
        "last_episode_to_air_episode_number","last_episode_to_air_production_code",
        "last_episode_to_air_runtime","last_episode_to_air_season_number",
        "last_episode_to_air_show_id","last_episode_to_air_still_path",
        "next_episode_to_air","networks","number_of_episodes","number_of_seasons",
        "origin_country","original_language","original_name","overview","popularity",
        "poster_path",
        "production_companies","production_countries","seasons","spoken_languages",
        "status","tagline","type_detail","vote_average","vote_count","review",
        "keyword","top_cast","directors","writers","providers"
    ]

    SEASON_COLS = [
        "_id","season_id","series_id","series_name","season_number","name","air_date",
        "overview","vote_average","vote_count","network_names","network_ids",
        "total_episodes","avg_episode_runtime","poster_path"
    ]

    df_series = df_series[SERIES_COLS].drop_duplicates(subset=["id"])
    if not df_seasons.empty:
        df_seasons = df_seasons[SEASON_COLS].drop_duplicates(subset=["series_id","season_number"])

    if not df_seasons.empty:
        df_series = fill_series_gaps(df_series, df_seasons)

    df_series.to_csv(SERIES_CSV, index=False, encoding="utf-8-sig")
    df_seasons.to_csv(SEASONS_CSV, index=False, encoding="utf-8-sig")

    try:
        df_series.to_parquet(SERIES_PARQ, index=False)
        df_seasons.to_parquet(SEASONS_PARQ, index=False)
    except:
        pass

    for f in [SERIES_TEMP, SEASONS_TEMP]:
        if f.exists():
            f.unlink()

    elapsed = (datetime.now() - t0).total_seconds()/60

    print("="*90)
    print("🎉 수집 완료")
    print("="*90)
    print(f"📌 Series 수: {len(df_series):,}개")
    print(f"📌 Season 수: {len(df_seasons):,}개")
    print(f"📌 총 API 요청: {stats['requests']:,}회")
    print(f"📌 에러 수: {stats['errors']:,}회")
    print(f"📌 평균 속도: {len(df_series)/elapsed:.1f} series/분")
    print(f"⏱ 총 소요시간: {elapsed:.1f}분")
    print("="*90)
    
    executor.shutdown(wait=False)

if __name__ == "__main__":
    asyncio.run(main())

✅ nest_asyncio 적용 완료
🚀 TMDB ULTRA-FAST SERIES + SEASONS COLLECTOR

📌 1단계: ID 수집
✨ 총 ID: 43,869개

📌 2단계: 상세 + 시즌 수집
⏱ 200/43,869 | 1.9 series/s | ETA: 385.7분 | 요청: 12,147 | 에러: 0
⏱ 400/43,869 | 3.8 series/s | ETA: 192.0분 | 요청: 12,147 | 에러: 0
⏱ 600/43,869 | 5.7 series/s | ETA: 127.4분 | 요청: 12,147 | 에러: 0
⏱ 800/43,869 | 7.5 series/s | ETA: 95.1분 | 요청: 12,147 | 에러: 0
⏱ 1,000/43,869 | 9.4 series/s | ETA: 75.7분 | 요청: 12,147 | 에러: 0
⏱ 1,200/43,869 | 11.3 series/s | ETA: 62.8분 | 요청: 12,147 | 에러: 0
⏱ 1,400/43,869 | 13.2 series/s | ETA: 53.6분 | 요청: 12,147 | 에러: 0
⏱ 1,600/43,869 | 15.1 series/s | ETA: 46.7분 | 요청: 12,147 | 에러: 0
⏱ 1,800/43,869 | 17.0 series/s | ETA: 41.3분 | 요청: 12,147 | 에러: 0
⏱ 2,000/43,869 | 18.9 series/s | ETA: 37.0분 | 요청: 12,147 | 에러: 0
⏱ 2,200/43,869 | 11.6 series/s | ETA: 60.0분 | 요청: 21,560 | 에러: 0
⏱ 2,400/43,869 | 12.6 series/s | ETA: 54.7분 | 요청: 21,560 | 에러: 0
⏱ 2,600/43,869 | 13.7 series/s | ETA: 50.3분 | 요청: 21,560 | 에러: 0
⏱ 2,800/43,869 | 14.7 series/s | ETA: 46.5분 | 요청: 

KeyboardInterrupt: 

In [ ]:
# ==========================================================
# TMDB TV SERIES + SEASONS HYPER-OPTIMIZED ASYNC COLLECTOR
# 비동기 방식 + 병목 제거 + 캐싱
# ==========================================================

import os
import asyncio
import aiohttp
from datetime import datetime, timedelta
from pathlib import Path
import pandas as pd
from dotenv import load_dotenv
from concurrent.futures import ThreadPoolExecutor
import time
from functools import lru_cache

# ==========================================================
# Jupyter async 지원
# ==========================================================
try:
    import nest_asyncio
    nest_asyncio.apply()
    print("✅ nest_asyncio 적용 완료")
except:
    pass

# ==========================================================
# 설정
# ==========================================================
load_dotenv()
API_KEY = os.getenv("API_KEY")
if not API_KEY:
    raise ValueError("API_KEY missing")

BASE_URL = "https://api.themoviedb.org/3"
HEADERS = {"accept": "application/json"}

# 🔥 핵심: TMDB 실제 제한은 50/초이지만, 버스트를 고려해 40으로 안정화
MAX_CALLS_PER_SECOND = 25
TIMEOUT = aiohttp.ClientTimeout(total=15, connect=5, sock_read=10)
MAX_RETRIES = 2

START_DATE = "2005-01-01"
END_DATE = "2015-12-31"

SERIES_TEMP = Path("tv_series_temp.csv")
SEASONS_TEMP = Path("tv_seasons_temp.csv")

SERIES_CSV = "tv_series_2005_2015_FULL.csv"
SEASONS_CSV = "tv_seasons_2005_2015_FULL.csv"
SERIES_PARQ = "tv_series_2005_2015_FULL.parquet"
SEASONS_PARQ = "tv_seasons_2005_2015_FULL.parquet"

# 통계
stats = {"requests": 0, "errors": 0, "retries": 0, "start_time": None}

# ==========================================================
# 고성능 Rate Limiter (Lock-free 버전)
# ==========================================================
class LockFreeRateLimiter:
    """Lock 없이 동작하는 초고속 Rate Limiter"""
    def __init__(self, rate):
        self.rate = rate
        self.tokens = rate
        self.updated_at = time.monotonic()
    
    async def acquire(self):
        now = time.monotonic()
        elapsed = now - self.updated_at
        
        # 토큰 보충 (Lock 없이)
        self.tokens = min(self.rate, self.tokens + elapsed * self.rate)
        self.updated_at = now
        
        # 토큰 부족시 최소 대기
        if self.tokens < 1:
            sleep_time = (1 - self.tokens) / self.rate
            await asyncio.sleep(sleep_time)
            self.tokens = 1
        
        self.tokens -= 1

rate_limiter = LockFreeRateLimiter(MAX_CALLS_PER_SECOND)

# ==========================================================
# TMDB GET (재시도 로직 개선)
# ==========================================================
async def tmdb_get(session, path, params=None, retry=0):
    if retry >= MAX_RETRIES:
        stats["errors"] += 1
        return None

    params = params or {}
    params.setdefault("api_key", API_KEY)
    params.setdefault("language", "en-US")

    url = f"{BASE_URL}{path}"
    
    await rate_limiter.acquire()
    stats["requests"] += 1

    try:
        async with session.get(url, params=params, headers=HEADERS) as resp:
            # 429: Rate limit
            if resp.status == 429:
                stats["retries"] += 1
                retry_after = int(resp.headers.get("Retry-After", 2))
                await asyncio.sleep(retry_after)
                return await tmdb_get(session, path, params, retry + 1)
            
            # 404: Not found (정상 케이스)
            if resp.status == 404:
                return None
            
            # 5xx: 서버 에러
            if 500 <= resp.status < 600:
                stats["retries"] += 1
                await asyncio.sleep(0.5 ** retry)  # 지수 감소
                return await tmdb_get(session, path, params, retry + 1)
            
            resp.raise_for_status()
            return await resp.json()
    
    except asyncio.TimeoutError:
        stats["errors"] += 1
        if retry < MAX_RETRIES - 1:
            await asyncio.sleep(0.2)
            return await tmdb_get(session, path, params, retry + 1)
        return None
    
    except Exception as e:
        stats["errors"] += 1
        if retry < MAX_RETRIES - 1:
            await asyncio.sleep(0.1)
            return await tmdb_get(session, path, params, retry + 1)
        return None

# ==========================================================
# Discover
# ==========================================================
async def fetch_discover_page(session, page, gte, lte):
    params = {
        "sort_by": "popularity.desc",
        "first_air_date.gte": gte,
        "first_air_date.lte": lte,
        "page": page,
        "include_adult": "true",
    }
    data = await tmdb_get(session, "/discover/tv", params)
    if not data:
        return [], 1, 0
    return data.get("results", []), data.get("total_pages", 1), data.get("total_results", 0)

# ==========================================================
# ID 수집 (개선)
# ==========================================================
async def collect_ids_in_range(session, start, end, depth=0):
    res1, total_pages, total_results = await fetch_discover_page(session, 1, start, end)

    if total_results == 0:
        return set()

    if total_pages <= 500:
        ids = {r["id"] for r in res1}
        
        if total_pages > 1:
            # 🔥 개선: 청크로 나눠서 메모리 효율 개선
            chunk_size = 100
            for chunk_start in range(2, total_pages + 1, chunk_size):
                chunk_end = min(chunk_start + chunk_size, total_pages + 1)
                results = await asyncio.gather(
                    *[fetch_discover_page(session, p, start, end) for p in range(chunk_start, chunk_end)],
                    return_exceptions=True
                )
                for r in results:
                    if isinstance(r, tuple):
                        ids.update(x["id"] for x in r[0])
        
        return ids

    # 날짜 분할
    start_dt = datetime.strptime(start, "%Y-%m-%d")
    end_dt = datetime.strptime(end, "%Y-%m-%d")
    mid_dt = start_dt + (end_dt - start_dt) / 2
    mid = mid_dt.strftime("%Y-%m-%d")
    right_start = (mid_dt + timedelta(days=1)).strftime("%Y-%m-%d")

    left, right = await asyncio.gather(
        collect_ids_in_range(session, start, mid, depth + 1),
        collect_ids_in_range(session, right_start, end, depth + 1)
    )
    return left | right

# ==========================================================
# Helpers (캐싱 추가)
# ==========================================================
@lru_cache(maxsize=1024)
def list_to_str_cached(lst_tuple, key="name"):
    """리스트를 문자열로 변환 (캐싱)"""
    if not lst_tuple:
        return ""
    return ", ".join(str(i.get(key, "")) for i in lst_tuple if i.get(key))

def list_to_str(lst, key="name"):
    if not lst:
        return ""
    # 딕셔너리 리스트를 튜플로 변환하여 캐싱 가능하게
    try:
        lst_tuple = tuple(tuple(d.items()) if isinstance(d, dict) else d for d in lst)
        return list_to_str_cached(lst_tuple, key)
    except:
        return ", ".join(str(i.get(key, "")) for i in lst if i.get(key))

def list_ids_to_str(lst, key="id"):
    if not lst:
        return ""
    return ", ".join(str(i.get(key)) for i in lst if i.get(key))

# ==========================================================
# Providers
# ==========================================================
async def fetch_providers(session, sid):
    data = await tmdb_get(session, f"/tv/{sid}/watch/providers")
    if not data:
        return ""
    out = set()
    results = data.get("results", {})
    for cc in ["US", "GB", "KR"]:
        if cc in results:
            for x in results[cc].get("flatrate", []):
                pn = x.get("provider_name")
                if pn:
                    out.add(pn)
    return ", ".join(sorted(out))

# ==========================================================
# Single Season
# ==========================================================
async def fetch_single_season(session, sid, sname, net_names, net_ids, meta):
    sn = meta.get("season_number")
    if sn is None:
        return None

    data = await tmdb_get(session, f"/tv/{sid}/season/{sn}")

    if not data:
        return {
            "_id": f"{sid}_{sn}",
            "season_id": meta.get("id"),
            "series_id": sid,
            "series_name": sname,
            "season_number": sn,
            "name": meta.get("name"),
            "air_date": meta.get("air_date"),
            "overview": meta.get("overview"),
            "vote_average": meta.get("vote_average"),
            "vote_count": meta.get("vote_count"),
            "network_names": net_names,
            "network_ids": net_ids,
            "total_episodes": meta.get("episode_count"),
            "avg_episode_runtime": None,
            "poster_path": meta.get("poster_path"),
        }

    eps = data.get("episodes") or []
    runtimes = [ep["runtime"] for ep in eps if ep.get("runtime")]
    avg_rt = sum(runtimes) / len(runtimes) if runtimes else None

    return {
        "_id": data.get("_id") or f"{sid}_{sn}",
        "season_id": data.get("id"),
        "series_id": sid,
        "series_name": sname,
        "season_number": data.get("season_number"),
        "name": data.get("name"),
        "air_date": data.get("air_date"),
        "overview": data.get("overview"),
        "vote_average": data.get("vote_average"),
        "vote_count": data.get("vote_count"),
        "network_names": net_names,
        "network_ids": net_ids,
        "total_episodes": len(eps),
        "avg_episode_runtime": avg_rt,
        "poster_path": data.get("poster_path"),
    }

# ==========================================================
# TV Details + Seasons
# ==========================================================
async def fetch_tv_details_and_seasons(session, sid):
    params = {"append_to_response": "reviews,keywords,aggregate_credits"}
    data = await tmdb_get(session, f"/tv/{sid}", params)
    if not data:
        return None, []

    genres = data.get("genres") or []
    networks = data.get("networks") or []
    last_ep = data.get("last_episode_to_air") or {}

    net_names = list_to_str(networks)
    net_ids = list_ids_to_str(networks)

    series = {
        "id": data.get("id"),
        "title": data.get("name"),
        "type": "tv_series",
        "adult": data.get("adult"),
        "backdrop_path": data.get("backdrop_path"),
        "created_by": list_to_str(data.get("created_by")),
        "episode_run_time": ", ".join(map(str, data.get("episode_run_time", []))) or "",
        "first_air_date": data.get("first_air_date"),
        "genres": list_to_str(genres),
        "genre_ids": ", ".join(str(g["id"]) for g in genres if g.get("id")),
        "homepage": data.get("homepage"),
        "in_production": data.get("in_production"),
        "languages": ", ".join(data.get("languages", [])) or "",
        "last_air_date": data.get("last_air_date"),
        "last_episode_to_air_id": last_ep.get("id"),
        "last_episode_to_air_name": last_ep.get("name"),
        "last_episode_to_air_overview": last_ep.get("overview"),
        "last_episode_to_air_vote_average": last_ep.get("vote_average"),
        "last_episode_to_air_vote_count": last_ep.get("vote_count"),
        "last_episode_to_air_air_date": last_ep.get("air_date"),
        "last_episode_to_air_episode_number": last_ep.get("episode_number"),
        "last_episode_to_air_production_code": last_ep.get("production_code"),
        "last_episode_to_air_runtime": last_ep.get("runtime"),
        "last_episode_to_air_season_number": last_ep.get("season_number"),
        "last_episode_to_air_show_id": last_ep.get("show_id"),
        "last_episode_to_air_still_path": last_ep.get("still_path"),
        "next_episode_to_air": str(data.get("next_episode_to_air")) if data.get("next_episode_to_air") else "",
        "networks": net_names,
        "number_of_episodes": data.get("number_of_episodes"),
        "number_of_seasons": data.get("number_of_seasons"),
        "origin_country": ", ".join(data.get("origin_country", [])) or "",
        "original_language": data.get("original_language"),
        "original_name": data.get("original_name"),
        "overview": data.get("overview"),
        "popularity": data.get("popularity"),
        "poster_path": data.get("poster_path"),
        "production_companies": list_to_str(data.get("production_companies")),
        "production_countries": list_to_str(data.get("production_countries")),
        "seasons": "; ".join(
            f"S{s['season_number']}: {s.get('name')} ({s.get('episode_count')} eps)"
            for s in (data.get("seasons") or []) if s.get("season_number") is not None
        ),
        "spoken_languages": list_to_str(data.get("spoken_languages")),
        "status": data.get("status"),
        "tagline": data.get("tagline"),
        "type_detail": data.get("type"),
        "vote_average": data.get("vote_average"),
        "vote_count": data.get("vote_count"),
    }

    # 리뷰
    rv_items = data.get("reviews", {}).get("results", [])
    series["review"] = " || ".join([
        f"{r.get('author', '')}({r.get('author_details', {}).get('rating', '')}): {(r.get('content') or '').replace(chr(10), ' ')[:200]}"
        for r in rv_items[:5]
    ])

    # 키워드
    kw_items = data.get("keywords", {}).get("results", [])
    series["keyword"] = ", ".join([k["name"] for k in kw_items[:20] if k.get("name")])

    # Cast & Crew
    credits = data.get("aggregate_credits", {})
    cast = credits.get("cast") or []
    crew = credits.get("crew") or []

    series["top_cast"] = ", ".join([c["name"] for c in cast[:4] if c.get("name")])

    dirs, wrs = set(), set()
    for c in crew:
        nm = c.get("name")
        if nm:
            for job in (c.get("jobs") or []):
                jn = job.get("job", "")
                if "Director" in jn:
                    dirs.add(nm)
                if jn in ["Writer", "Screenplay", "Story"]:
                    wrs.add(nm)

    series["directors"] = ", ".join(sorted(dirs)[:10])
    series["writers"] = ", ".join(sorted(wrs)[:10])

    # 🔥 providers + seasons 병렬 수집
    seasons_meta = [s for s in (data.get("seasons") or []) if s.get("season_number") is not None]
    
    tasks = [fetch_providers(session, sid)]
    tasks.extend([
        fetch_single_season(session, sid, data.get("name", ""), net_names, net_ids, s)
        for s in seasons_meta
    ])
    
    results = await asyncio.gather(*tasks, return_exceptions=True)
    
    series["providers"] = results[0] if isinstance(results[0], str) else ""
    season_records = [r for r in results[1:] if r and not isinstance(r, Exception)]

    return series, season_records

# ==========================================================
# 비동기 CSV 쓰기
# ==========================================================
executor = ThreadPoolExecutor(max_workers=2)

async def save_to_csv_async(df, path, mode, header):
    loop = asyncio.get_event_loop()
    await loop.run_in_executor(
        executor,
        lambda: df.to_csv(path, mode=mode, header=header, index=False, encoding="utf-8-sig")
    )

# ==========================================================
# 결측치 보완
# ==========================================================
def fill_series_gaps(series_df, seasons_df):
    print("\n📊 시즌 기반 결측치 보완")

    rtmap = seasons_df.groupby("series_id")["avg_episode_runtime"].mean().round()
    mask = series_df["episode_run_time"].astype(str).str.strip().isin(["", "nan"])
    series_df.loc[mask, "episode_run_time"] = (
        series_df.loc[mask, "id"].map(rtmap).fillna(0).astype(int).astype(str)
    )

    epmap = seasons_df.groupby("series_id")["total_episodes"].sum()
    mask2 = series_df["number_of_episodes"].isna()
    series_df.loc[mask2, "number_of_episodes"] = series_df.loc[mask2, "id"].map(epmap)

    seasmap = seasons_df.groupby("series_id")["season_number"].nunique()
    mask3 = series_df["number_of_seasons"].isna()
    series_df.loc[mask3, "number_of_seasons"] = series_df.loc[mask3, "id"].map(seasmap)

    lastair = seasons_df.sort_values("air_date").groupby("series_id")["air_date"].last()
    ladmask = series_df["last_air_date"].astype(str).str.strip().isin(["", "nan", "NaT"])
    series_df.loc[ladmask, "last_air_date"] = series_df.loc[ladmask, "id"].map(lastair)

    print("✅ 완료\n")
    return series_df

# ==========================================================
# MAIN
# ==========================================================
async def main():
    print("=" * 90)
    print("🚀 TMDB HYPER-OPTIMIZED ASYNC COLLECTOR")
    print("=" * 90)
    stats["start_time"] = datetime.now()
    t0 = datetime.now()

    for f in [SERIES_TEMP, SEASONS_TEMP]:
        if f.exists():
            f.unlink()

    # 🔥 최적 설정
    connector = aiohttp.TCPConnector(
        limit=150,
        limit_per_host=75,
        ttl_dns_cache=600,
        force_close=False,
        enable_cleanup_closed=True
    )

    async with aiohttp.ClientSession(connector=connector, timeout=TIMEOUT) as session:
        # 1) ID 수집
        print("\n📌 1단계: ID 수집")
        ids = sorted(list(await collect_ids_in_range(session, START_DATE, END_DATE)))
        print(f"✨ 총 ID: {len(ids):,}개")

        # 2) 상세 수집
        print("\n📌 2단계: 상세 + 시즌 수집")
        batch_size = 1500
        sem = asyncio.Semaphore(30)  # 🔥 안정적인 동시성

        async def fetch_one(i):
            async with sem:
                return await fetch_tv_details_and_seasons(session, i)

        processed = 0
        last_print = time.time()
        
        for start_idx in range(0, len(ids), batch_size):
            batch_start = time.time()
            part = ids[start_idx:start_idx+batch_size]
            
            results = await asyncio.gather(*[fetch_one(x) for x in part], return_exceptions=True)
            
            bs, bse = [], []
            for r in results:
                if isinstance(r, tuple) and r[0]:
                    s, ss = r
                    bs.append(s)
                    bse.extend(ss)
                processed += 1

            # CSV 쓰기
            if bs:
                await save_to_csv_async(pd.DataFrame(bs), SERIES_TEMP, "a", not SERIES_TEMP.exists())
            if bse:
                await save_to_csv_async(pd.DataFrame(bse), SEASONS_TEMP, "a", not SEASONS_TEMP.exists())

            # 진행 상황 (5초마다)
            now = time.time()
            if now - last_print >= 5:
                elapsed = (datetime.now() - t0).total_seconds()
                rate = processed / elapsed if elapsed > 0 else 0
                eta = (len(ids) - processed) / rate / 60 if rate > 0 else 0
                batch_time = now - batch_start
                print(f"⏱ {processed:,}/{len(ids):,} ({processed/len(ids)*100:.1f}%) | "
                      f"{rate:.1f}/s | ETA: {eta:.0f}분 | Batch: {batch_time:.1f}s | "
                      f"요청: {stats['requests']:,} | 에러: {stats['errors']} | 재시도: {stats['retries']}")
                last_print = now

    # 3) 최종 처리
    print("\n📌 3단계: 최종 처리")
    df_series = pd.read_csv(SERIES_TEMP)
    df_seasons = pd.read_csv(SEASONS_TEMP) if SEASONS_TEMP.exists() else pd.DataFrame()

    SERIES_COLS = [
        "id","title","type","adult","backdrop_path","created_by","episode_run_time",
        "first_air_date","genres","genre_ids","homepage","in_production","languages",
        "last_air_date","last_episode_to_air_id","last_episode_to_air_name",
        "last_episode_to_air_overview","last_episode_to_air_vote_average",
        "last_episode_to_air_vote_count","last_episode_to_air_air_date",
        "last_episode_to_air_episode_number","last_episode_to_air_production_code",
        "last_episode_to_air_runtime","last_episode_to_air_season_number",
        "last_episode_to_air_show_id","last_episode_to_air_still_path",
        "next_episode_to_air","networks","number_of_episodes","number_of_seasons",
        "origin_country","original_language","original_name","overview","popularity",
        "poster_path","production_companies","production_countries","seasons",
        "spoken_languages","status","tagline","type_detail","vote_average",
        "vote_count","review","keyword","top_cast","directors","writers","providers"
    ]

    SEASON_COLS = [
        "_id","season_id","series_id","series_name","season_number","name","air_date",
        "overview","vote_average","vote_count","network_names","network_ids",
        "total_episodes","avg_episode_runtime","poster_path"
    ]

    df_series = df_series[SERIES_COLS].drop_duplicates(subset=["id"])
    if not df_seasons.empty:
        df_seasons = df_seasons[SEASON_COLS].drop_duplicates(subset=["series_id","season_number"])
        df_series = fill_series_gaps(df_series, df_seasons)

    df_series.to_csv(SERIES_CSV, index=False, encoding="utf-8-sig")
    df_seasons.to_csv(SEASONS_CSV, index=False, encoding="utf-8-sig")

    try:
        df_series.to_parquet(SERIES_PARQ, index=False)
        df_seasons.to_parquet(SEASONS_PARQ, index=False)
    except:
        pass

    for f in [SERIES_TEMP, SEASONS_TEMP]:
        if f.exists():
            f.unlink()

    elapsed = (datetime.now() - t0).total_seconds()/60

    print("="*90)
    print("🎉 수집 완료!")
    print("="*90)
    print(f"📌 Series: {len(df_series):,}개 | Seasons: {len(df_seasons):,}개")
    print(f"📌 API 요청: {stats['requests']:,}회 | 에러: {stats['errors']} | 재시도: {stats['retries']}")
    print(f"📌 평균 속도: {len(df_series)/elapsed:.1f} series/분")
    print(f"⏱ 총 시간: {elapsed:.1f}분 ({elapsed/60:.1f}시간)")
    print("="*90)
    
    executor.shutdown(wait=False)

if __name__ == "__main__":
    asyncio.run(main())

✅ nest_asyncio 적용 완료
🚀 TMDB HYPER-OPTIMIZED ASYNC COLLECTOR

📌 1단계: ID 수집
✨ 총 ID: 43,869개

📌 2단계: 상세 + 시즌 수집
⏱ 1,500/43,869 (3.4%) | 48.7/s | ETA: 15분 | Batch: 23.9s | 요청: 9,916 | 에러: 0 | 재시도: 0
⏱ 3,000/43,869 (6.8%) | 59.4/s | ETA: 11분 | Batch: 19.7s | 요청: 16,700 | 에러: 0 | 재시도: 0
⏱ 4,500/43,869 (10.3%) | 62.8/s | ETA: 10분 | Batch: 21.1s | 요청: 23,488 | 에러: 0 | 재시도: 0
⏱ 6,000/43,869 (13.7%) | 65.2/s | ETA: 10분 | Batch: 20.4s | 요청: 29,619 | 에러: 0 | 재시도: 0
⏱ 7,500/43,869 (17.1%) | 66.8/s | ETA: 9분 | Batch: 20.2s | 요청: 36,695 | 에러: 0 | 재시도: 0
⏱ 9,000/43,869 (20.5%) | 68.3/s | ETA: 9분 | Batch: 19.6s | 요청: 42,946 | 에러: 0 | 재시도: 0
⏱ 10,500/43,869 (23.9%) | 69.0/s | ETA: 8분 | Batch: 20.4s | 요청: 49,881 | 에러: 0 | 재시도: 0
⏱ 12,000/43,869 (27.4%) | 69.4/s | ETA: 8분 | Batch: 20.8s | 요청: 56,557 | 에러: 0 | 재시도: 0
⏱ 13,500/43,869 (30.8%) | 69.9/s | ETA: 7분 | Batch: 20.2s | 요청: 63,094 | 에러: 0 | 재시도: 0
⏱ 15,000/43,869 (34.2%) | 70.2/s | ETA: 7분 | Batch: 20.3s | 요청: 69,740 | 에러: 0 | 재시도: 0
⏱ 16,500/43,869 

In [25]:
# ==========================================================
# TMDB TV SERIES + SEASONS HYPER-OPTIMIZED ASYNC COLLECTOR
# 비동기 방식 + 병목 제거 + 캐싱
# ✨ 최종 개선: providers를 flatrate/rent/buy로 분리 + imdb_id 추가
# ==========================================================

import os
import asyncio
import aiohttp
from datetime import datetime, timedelta
from pathlib import Path
import pandas as pd
from dotenv import load_dotenv
from concurrent.futures import ThreadPoolExecutor
import time
from functools import lru_cache
import json

# ==========================================================
# Jupyter async 지원
# ==========================================================
try:
    import nest_asyncio
    nest_asyncio.apply()
    print("✅ nest_asyncio 적용 완료")
except:
    pass

# ==========================================================
# 설정
# ==========================================================
load_dotenv()
API_KEY = os.getenv("API_KEY")
if not API_KEY:
    raise ValueError("API_KEY missing")

BASE_URL = "https://api.themoviedb.org/3"
HEADERS = {"accept": "application/json"}

# 🔥 핵심: TMDB 실제 제한은 50/초이지만, 버스트를 고려해 40으로 안정화
MAX_CALLS_PER_SECOND = 25
TIMEOUT = aiohttp.ClientTimeout(total=15, connect=5, sock_read=10)
MAX_RETRIES = 2

START_DATE = "2005-01-01"
END_DATE = "2015-12-31"

SERIES_TEMP = Path("tv_series_temp.csv")
SEASONS_TEMP = Path("tv_seasons_temp.csv")

SERIES_CSV = "tv_series_2005_2015_FULL.csv"
SEASONS_CSV = "tv_seasons_2005_2015_FULL.csv"
SERIES_PARQ = "tv_series_2005_2015_FULL.parquet"
SEASONS_PARQ = "tv_seasons_2005_2015_FULL.parquet"

# 통계
stats = {"requests": 0, "errors": 0, "retries": 0, "start_time": None}

# ==========================================================
# 고성능 Rate Limiter (Lock-free 버전)
# ==========================================================
class LockFreeRateLimiter:
    """Lock 없이 동작하는 초고속 Rate Limiter"""
    def __init__(self, rate):
        self.rate = rate
        self.tokens = rate
        self.updated_at = time.monotonic()
    
    async def acquire(self):
        now = time.monotonic()
        elapsed = now - self.updated_at
        
        # 토큰 보충 (Lock 없이)
        self.tokens = min(self.rate, self.tokens + elapsed * self.rate)
        self.updated_at = now
        
        # 토큰 부족시 최소 대기
        if self.tokens < 1:
            sleep_time = (1 - self.tokens) / self.rate
            await asyncio.sleep(sleep_time)
            self.tokens = 1
        
        self.tokens -= 1

rate_limiter = LockFreeRateLimiter(MAX_CALLS_PER_SECOND)

# ==========================================================
# TMDB GET (재시도 로직 개선)
# ==========================================================
async def tmdb_get(session, path, params=None, retry=0):
    if retry >= MAX_RETRIES:
        stats["errors"] += 1
        return None

    params = params or {}
    params.setdefault("api_key", API_KEY)
    params.setdefault("language", "en-US")

    url = f"{BASE_URL}{path}"
    
    await rate_limiter.acquire()
    stats["requests"] += 1

    try:
        async with session.get(url, params=params, headers=HEADERS) as resp:
            # 429: Rate limit
            if resp.status == 429:
                stats["retries"] += 1
                retry_after = int(resp.headers.get("Retry-After", 2))
                await asyncio.sleep(retry_after)
                return await tmdb_get(session, path, params, retry + 1)
            
            # 404: Not found (정상 케이스)
            if resp.status == 404:
                return None
            
            # 5xx: 서버 에러
            if 500 <= resp.status < 600:
                stats["retries"] += 1
                await asyncio.sleep(0.5 ** retry)  # 지수 감소
                return await tmdb_get(session, path, params, retry + 1)
            
            resp.raise_for_status()
            return await resp.json()
    
    except asyncio.TimeoutError:
        stats["errors"] += 1
        if retry < MAX_RETRIES - 1:
            await asyncio.sleep(0.2)
            return await tmdb_get(session, path, params, retry + 1)
        return None
    
    except Exception as e:
        stats["errors"] += 1
        if retry < MAX_RETRIES - 1:
            await asyncio.sleep(0.1)
            return await tmdb_get(session, path, params, retry + 1)
        return None

# ==========================================================
# Discover
# ==========================================================
async def fetch_discover_page(session, page, gte, lte):
    params = {
        "sort_by": "popularity.desc",
        "first_air_date.gte": gte,
        "first_air_date.lte": lte,
        "page": page,
        "include_adult": "true",
    }
    data = await tmdb_get(session, "/discover/tv", params)
    if not data:
        return [], 1, 0
    return data.get("results", []), data.get("total_pages", 1), data.get("total_results", 0)

# ==========================================================
# ID 수집 (개선)
# ==========================================================
async def collect_ids_in_range(session, start, end, depth=0):
    res1, total_pages, total_results = await fetch_discover_page(session, 1, start, end)

    if total_results == 0:
        return set()

    if total_pages <= 500:
        ids = {r["id"] for r in res1}
        
        if total_pages > 1:
            # 🔥 개선: 청크로 나눠서 메모리 효율 개선
            chunk_size = 100
            for chunk_start in range(2, total_pages + 1, chunk_size):
                chunk_end = min(chunk_start + chunk_size, total_pages + 1)
                results = await asyncio.gather(
                    *[fetch_discover_page(session, p, start, end) for p in range(chunk_start, chunk_end)],
                    return_exceptions=True
                )
                for r in results:
                    if isinstance(r, tuple):
                        ids.update(x["id"] for x in r[0])
        
        return ids

    # 날짜 분할
    start_dt = datetime.strptime(start, "%Y-%m-%d")
    end_dt = datetime.strptime(end, "%Y-%m-%d")
    mid_dt = start_dt + (end_dt - start_dt) / 2
    mid = mid_dt.strftime("%Y-%m-%d")
    right_start = (mid_dt + timedelta(days=1)).strftime("%Y-%m-%d")

    left, right = await asyncio.gather(
        collect_ids_in_range(session, start, mid, depth + 1),
        collect_ids_in_range(session, right_start, end, depth + 1)
    )
    return left | right

# ==========================================================
# Helpers (캐싱 추가)
# ==========================================================
@lru_cache(maxsize=1024)
def list_to_str_cached(lst_tuple, key="name"):
    """리스트를 문자열로 변환 (캐싱)"""
    if not lst_tuple:
        return ""
    return ", ".join(str(i.get(key, "")) for i in lst_tuple if i.get(key))

def list_to_str(lst, key="name"):
    if not lst:
        return ""
    # 딕셔너리 리스트를 튜플로 변환하여 캐싱 가능하게
    try:
        lst_tuple = tuple(tuple(d.items()) if isinstance(d, dict) else d for d in lst)
        return list_to_str_cached(lst_tuple, key)
    except:
        return ", ".join(str(i.get(key, "")) for i in lst if i.get(key))

def list_ids_to_str(lst, key="id"):
    if not lst:
        return ""
    return ", ".join(str(i.get(key)) for i in lst if i.get(key))

# ==========================================================
# ✨ NEW: External IDs (IMDB ID)
# ==========================================================
async def fetch_external_ids(session, sid):
    """시리즈의 external IDs를 가져와서 IMDB ID 반환"""
    data = await tmdb_get(session, f"/tv/{sid}/external_ids")
    if not data:
        return ""
    return data.get("imdb_id", "")

# ==========================================================
# ✨ UPDATED: Providers (flatrate/rent/buy 분리)
# ==========================================================
async def fetch_providers(session, sid):
    """
    모든 국가의 streaming providers 수집
    flatrate(구독형), rent(대여), buy(구매) 타입별로 분리
    반환: (flatrate_dict, rent_dict, buy_dict)
    """
    data = await tmdb_get(session, f"/tv/{sid}/watch/providers")
    if not data:
        return "{}", "{}", "{}"
    
    providers_results = data.get("results", {})
    
    # 국가별 OTT 정보 수집
    providers_flatrate = {}
    providers_rent = {}
    providers_buy = {}
    
    for country, info in providers_results.items():
        # flatrate: 구독형 (Netflix, Disney+ 등)
        flatrate = info.get("flatrate", [])
        if flatrate:
            provider_names = [p.get("provider_name") for p in flatrate if p.get("provider_name")]
            if provider_names:
                providers_flatrate[country] = provider_names
        
        # rent: 대여
        rent = info.get("rent", [])
        if rent:
            provider_names = [p.get("provider_name") for p in rent if p.get("provider_name")]
            if provider_names:
                providers_rent[country] = provider_names
        
        # buy: 구매
        buy = info.get("buy", [])
        if buy:
            provider_names = [p.get("provider_name") for p in buy if p.get("provider_name")]
            if provider_names:
                providers_buy[country] = provider_names
    
    # JSON 문자열로 변환하여 반환
    return (
        json.dumps(providers_flatrate, ensure_ascii=False) if providers_flatrate else "{}",
        json.dumps(providers_rent, ensure_ascii=False) if providers_rent else "{}",
        json.dumps(providers_buy, ensure_ascii=False) if providers_buy else "{}"
    )

# ==========================================================
# Single Season
# ==========================================================
async def fetch_single_season(session, sid, sname, net_names, net_ids, meta):
    sn = meta.get("season_number")
    if sn is None:
        return None

    data = await tmdb_get(session, f"/tv/{sid}/season/{sn}")

    if not data:
        return {
            "_id": f"{sid}_{sn}",
            "season_id": meta.get("id"),
            "series_id": sid,
            "series_name": sname,
            "season_number": sn,
            "name": meta.get("name"),
            "air_date": meta.get("air_date"),
            "overview": meta.get("overview"),
            "vote_average": meta.get("vote_average"),
            "vote_count": meta.get("vote_count"),
            "network_names": net_names,
            "network_ids": net_ids,
            "total_episodes": meta.get("episode_count"),
            "avg_episode_runtime": None,
            "poster_path": meta.get("poster_path"),
        }

    eps = data.get("episodes") or []
    runtimes = [ep["runtime"] for ep in eps if ep.get("runtime")]
    avg_rt = sum(runtimes) / len(runtimes) if runtimes else None

    return {
        "_id": data.get("_id") or f"{sid}_{sn}",
        "season_id": data.get("id"),
        "series_id": sid,
        "series_name": sname,
        "season_number": data.get("season_number"),
        "name": data.get("name"),
        "air_date": data.get("air_date"),
        "overview": data.get("overview"),
        "vote_average": data.get("vote_average"),
        "vote_count": data.get("vote_count"),
        "network_names": net_names,
        "network_ids": net_ids,
        "total_episodes": len(eps),
        "avg_episode_runtime": avg_rt,
        "poster_path": data.get("poster_path"),
    }

# ==========================================================
# ✨ UPDATED: TV Details + Seasons + IMDB ID + Providers
# ==========================================================
async def fetch_tv_details_and_seasons(session, sid):
    params = {"append_to_response": "reviews,keywords,aggregate_credits"}
    data = await tmdb_get(session, f"/tv/{sid}", params)
    if not data:
        return None, []

    genres = data.get("genres") or []
    networks = data.get("networks") or []
    last_ep = data.get("last_episode_to_air") or {}

    net_names = list_to_str(networks)
    net_ids = list_ids_to_str(networks)

    series = {
        "id": data.get("id"),
        "title": data.get("name"),
        "type": "tv_series",
        "adult": data.get("adult"),
        "backdrop_path": data.get("backdrop_path"),
        "created_by": list_to_str(data.get("created_by")),
        "episode_run_time": ", ".join(map(str, data.get("episode_run_time", []))) or "",
        "first_air_date": data.get("first_air_date"),
        "genres": list_to_str(genres),
        "genre_ids": ", ".join(str(g["id"]) for g in genres if g.get("id")),
        "homepage": data.get("homepage"),
        "in_production": data.get("in_production"),
        "languages": ", ".join(data.get("languages", [])) or "",
        "last_air_date": data.get("last_air_date"),
        "last_episode_to_air_id": last_ep.get("id"),
        "last_episode_to_air_name": last_ep.get("name"),
        "last_episode_to_air_overview": last_ep.get("overview"),
        "last_episode_to_air_vote_average": last_ep.get("vote_average"),
        "last_episode_to_air_vote_count": last_ep.get("vote_count"),
        "last_episode_to_air_air_date": last_ep.get("air_date"),
        "last_episode_to_air_episode_number": last_ep.get("episode_number"),
        "last_episode_to_air_production_code": last_ep.get("production_code"),
        "last_episode_to_air_runtime": last_ep.get("runtime"),
        "last_episode_to_air_season_number": last_ep.get("season_number"),
        "last_episode_to_air_show_id": last_ep.get("show_id"),
        "last_episode_to_air_still_path": last_ep.get("still_path"),
        "next_episode_to_air": str(data.get("next_episode_to_air")) if data.get("next_episode_to_air") else "",
        "networks": net_names,
        "number_of_episodes": data.get("number_of_episodes"),
        "number_of_seasons": data.get("number_of_seasons"),
        "origin_country": ", ".join(data.get("origin_country", [])) or "",
        "original_language": data.get("original_language"),
        "original_name": data.get("original_name"),
        "overview": data.get("overview"),
        "popularity": data.get("popularity"),
        "poster_path": data.get("poster_path"),
        "production_companies": list_to_str(data.get("production_companies")),
        "production_countries": list_to_str(data.get("production_countries")),
        "seasons": "; ".join(
            f"S{s['season_number']}: {s.get('name')} ({s.get('episode_count')} eps)"
            for s in (data.get("seasons") or []) if s.get("season_number") is not None
        ),
        "spoken_languages": list_to_str(data.get("spoken_languages")),
        "status": data.get("status"),
        "tagline": data.get("tagline"),
        "type_detail": data.get("type"),
        "vote_average": data.get("vote_average"),
        "vote_count": data.get("vote_count"),
    }

    # 리뷰
    rv_items = data.get("reviews", {}).get("results", [])
    series["review"] = " || ".join([
        f"{r.get('author', '')}({r.get('author_details', {}).get('rating', '')}): {(r.get('content') or '').replace(chr(10), ' ')[:200]}"
        for r in rv_items[:5]
    ])

    # 키워드
    kw_items = data.get("keywords", {}).get("results", [])
    series["keyword"] = ", ".join([k["name"] for k in kw_items[:20] if k.get("name")])

    # Cast & Crew
    credits = data.get("aggregate_credits", {})
    cast = credits.get("cast") or []
    crew = credits.get("crew") or []

    series["top_cast"] = ", ".join([c["name"] for c in cast[:4] if c.get("name")])

    dirs, wrs = set(), set()
    for c in crew:
        nm = c.get("name")
        if nm:
            for job in (c.get("jobs") or []):
                jn = job.get("job", "")
                if "Director" in jn:
                    dirs.add(nm)
                if jn in ["Writer", "Screenplay", "Story"]:
                    wrs.add(nm)

    series["directors"] = ", ".join(sorted(dirs)[:10])
    series["writers"] = ", ".join(sorted(wrs)[:10])

    # 🔥 providers + external_ids + seasons 병렬 수집
    seasons_meta = [s for s in (data.get("seasons") or []) if s.get("season_number") is not None]
    
    tasks = [
        fetch_providers(session, sid),      # 0: (flatrate, rent, buy)
        fetch_external_ids(session, sid),   # 1: imdb_id
    ]
    tasks.extend([
        fetch_single_season(session, sid, data.get("name", ""), net_names, net_ids, s)
        for s in seasons_meta
    ])
    
    results = await asyncio.gather(*tasks, return_exceptions=True)
    
    # Providers 처리 (flatrate, rent, buy)
    if isinstance(results[0], tuple) and len(results[0]) == 3:
        series["providers_flatrate"] = results[0][0]
        series["providers_rent"] = results[0][1]
        series["providers_buy"] = results[0][2]
    else:
        series["providers_flatrate"] = "{}"
        series["providers_rent"] = "{}"
        series["providers_buy"] = "{}"
    
    # IMDB ID 처리
    series["imdb_id"] = results[1] if isinstance(results[1], str) else ""
    
    # Seasons 처리
    season_records = [r for r in results[2:] if r and not isinstance(r, Exception)]

    return series, season_records

# ==========================================================
# 비동기 CSV 쓰기
# ==========================================================
executor = ThreadPoolExecutor(max_workers=2)

async def save_to_csv_async(df, path, mode, header):
    loop = asyncio.get_event_loop()
    await loop.run_in_executor(
        executor,
        lambda: df.to_csv(path, mode=mode, header=header, index=False, encoding="utf-8-sig")
    )

# ==========================================================
# 결측치 보완
# ==========================================================
def fill_series_gaps(series_df, seasons_df):
    print("\n📊 시즌 기반 결측치 보완")

    rtmap = seasons_df.groupby("series_id")["avg_episode_runtime"].mean().round()
    mask = series_df["episode_run_time"].astype(str).str.strip().isin(["", "nan"])
    series_df.loc[mask, "episode_run_time"] = (
        series_df.loc[mask, "id"].map(rtmap).fillna(0).astype(int).astype(str)
    )

    epmap = seasons_df.groupby("series_id")["total_episodes"].sum()
    mask2 = series_df["number_of_episodes"].isna()
    series_df.loc[mask2, "number_of_episodes"] = series_df.loc[mask2, "id"].map(epmap)

    seasmap = seasons_df.groupby("series_id")["season_number"].nunique()
    mask3 = series_df["number_of_seasons"].isna()
    series_df.loc[mask3, "number_of_seasons"] = series_df.loc[mask3, "id"].map(seasmap)

    lastair = seasons_df.sort_values("air_date").groupby("series_id")["air_date"].last()
    ladmask = series_df["last_air_date"].astype(str).str.strip().isin(["", "nan", "NaT"])
    series_df.loc[ladmask, "last_air_date"] = series_df.loc[ladmask, "id"].map(lastair)

    print("✅ 완료\n")
    return series_df

# ==========================================================
# MAIN
# ==========================================================
async def main():
    print("=" * 90)
    print("🚀 TMDB HYPER-OPTIMIZED ASYNC COLLECTOR")
    print("✨ 최종 개선: providers(flatrate/rent/buy 분리) + IMDB ID 추가")
    print("=" * 90)
    stats["start_time"] = datetime.now()
    t0 = datetime.now()

    for f in [SERIES_TEMP, SEASONS_TEMP]:
        if f.exists():
            f.unlink()

    # 🔥 최적 설정
    connector = aiohttp.TCPConnector(
        limit=150,
        limit_per_host=75,
        ttl_dns_cache=600,
        force_close=False,
        enable_cleanup_closed=True
    )

    async with aiohttp.ClientSession(connector=connector, timeout=TIMEOUT) as session:
        # 1) ID 수집
        print("\n📌 1단계: ID 수집")
        ids = sorted(list(await collect_ids_in_range(session, START_DATE, END_DATE)))
        print(f"✨ 총 ID: {len(ids):,}개")

        # 2) 상세 수집
        print("\n📌 2단계: 상세 + 시즌 + IMDB ID + Providers 수집")
        batch_size = 1500
        sem = asyncio.Semaphore(30)  # 🔥 안정적인 동시성

        async def fetch_one(i):
            async with sem:
                return await fetch_tv_details_and_seasons(session, i)

        processed = 0
        last_print = time.time()
        
        for start_idx in range(0, len(ids), batch_size):
            batch_start = time.time()
            part = ids[start_idx:start_idx+batch_size]
            
            results = await asyncio.gather(*[fetch_one(x) for x in part], return_exceptions=True)
            
            bs, bse = [], []
            for r in results:
                if isinstance(r, tuple) and r[0]:
                    s, ss = r
                    bs.append(s)
                    bse.extend(ss)
                processed += 1

            # CSV 쓰기
            if bs:
                await save_to_csv_async(pd.DataFrame(bs), SERIES_TEMP, "a", not SERIES_TEMP.exists())
            if bse:
                await save_to_csv_async(pd.DataFrame(bse), SEASONS_TEMP, "a", not SEASONS_TEMP.exists())

            # 진행 상황 (5초마다)
            now = time.time()
            if now - last_print >= 5:
                elapsed = (datetime.now() - t0).total_seconds()
                rate = processed / elapsed if elapsed > 0 else 0
                eta = (len(ids) - processed) / rate / 60 if rate > 0 else 0
                batch_time = now - batch_start
                print(f"⏱ {processed:,}/{len(ids):,} ({processed/len(ids)*100:.1f}%) | "
                      f"{rate:.1f}/s | ETA: {eta:.0f}분 | Batch: {batch_time:.1f}s | "
                      f"요청: {stats['requests']:,} | 에러: {stats['errors']} | 재시도: {stats['retries']}")
                last_print = now

    # 3) 최종 처리
    print("\n📌 3단계: 최종 처리")
    df_series = pd.read_csv(SERIES_TEMP)
    df_seasons = pd.read_csv(SEASONS_TEMP) if SEASONS_TEMP.exists() else pd.DataFrame()

    SERIES_COLS = [
        "id","title","type","adult","backdrop_path","created_by","episode_run_time",
        "first_air_date","genres","genre_ids","homepage","in_production","languages",
        "last_air_date","last_episode_to_air_id","last_episode_to_air_name",
        "last_episode_to_air_overview","last_episode_to_air_vote_average",
        "last_episode_to_air_vote_count","last_episode_to_air_air_date",
        "last_episode_to_air_episode_number","last_episode_to_air_production_code",
        "last_episode_to_air_runtime","last_episode_to_air_season_number",
        "last_episode_to_air_show_id","last_episode_to_air_still_path",
        "next_episode_to_air","networks","number_of_episodes","number_of_seasons",
        "origin_country","original_language","original_name","overview","popularity",
        "poster_path","production_companies","production_countries","seasons",
        "spoken_languages","status","tagline","type_detail","vote_average",
        "vote_count","review","keyword","top_cast","directors","writers",
        "providers_flatrate","providers_rent","providers_buy","imdb_id"  # ✨ 변경된 컬럼들
    ]

    SEASON_COLS = [
        "_id","season_id","series_id","series_name","season_number","name","air_date",
        "overview","vote_average","vote_count","network_names","network_ids",
        "total_episodes","avg_episode_runtime","poster_path"
    ]

    df_series = df_series[SERIES_COLS].drop_duplicates(subset=["id"])
    if not df_seasons.empty:
        df_seasons = df_seasons[SEASON_COLS].drop_duplicates(subset=["series_id","season_number"])
        df_series = fill_series_gaps(df_series, df_seasons)

    df_series.to_csv(SERIES_CSV, index=False, encoding="utf-8-sig")
    df_seasons.to_csv(SEASONS_CSV, index=False, encoding="utf-8-sig")

    try:
        df_series.to_parquet(SERIES_PARQ, index=False)
        df_seasons.to_parquet(SEASONS_PARQ, index=False)
    except:
        pass

    for f in [SERIES_TEMP, SEASONS_TEMP]:
        if f.exists():
            f.unlink()

    elapsed = (datetime.now() - t0).total_seconds()/60

    print("="*90)
    print("🎉 수집 완료!")
    print("="*90)
    print(f"📌 Series: {len(df_series):,}개 | Seasons: {len(df_seasons):,}개")
    print(f"📌 API 요청: {stats['requests']:,}회 | 에러: {stats['errors']} | 재시도: {stats['retries']}")
    print(f"📌 평균 속도: {len(df_series)/elapsed:.1f} series/분")
    print(f"⏱ 총 시간: {elapsed:.1f}분 ({elapsed/60:.1f}시간)")
    print("="*90)
    
    # 샘플 데이터 출력
    print("\n📊 샘플 Providers 데이터:")
    if not df_series.empty and 'providers_flatrate' in df_series.columns:
        sample = df_series[df_series['providers_flatrate'] != '{}'].head(1)
        if not sample.empty:
            print(f"Title: {sample.iloc[0]['title']}")
            print(f"Flatrate: {sample.iloc[0]['providers_flatrate'][:200]}...")
            print(f"IMDB ID: {sample.iloc[0]['imdb_id']}")
    
    executor.shutdown(wait=False)

if __name__ == "__main__":
    asyncio.run(main())

✅ nest_asyncio 적용 완료
🚀 TMDB HYPER-OPTIMIZED ASYNC COLLECTOR
✨ 최종 개선: providers(flatrate/rent/buy 분리) + IMDB ID 추가

📌 1단계: ID 수집
✨ 총 ID: 37,971개

📌 2단계: 상세 + 시즌 + IMDB ID + Providers 수집
⏱ 1,500/37,971 (4.0%) | 27.2/s | ETA: 22분 | Batch: 47.1s | 요청: 11,344 | 에러: 0 | 재시도: 0
⏱ 3,000/37,971 (7.9%) | 14.1/s | ETA: 41분 | Batch: 158.1s | 요청: 19,974 | 에러: 0 | 재시도: 36
⏱ 4,500/37,971 (11.9%) | 12.9/s | ETA: 43분 | Batch: 136.8s | 요청: 27,772 | 에러: 0 | 재시도: 38
⏱ 6,000/37,971 (15.8%) | 11.8/s | ETA: 45분 | Batch: 159.1s | 요청: 36,247 | 에러: 0 | 재시도: 42
⏱ 7,500/37,971 (19.8%) | 11.5/s | ETA: 44분 | Batch: 145.4s | 요청: 44,155 | 에러: 0 | 재시도: 48
⏱ 9,000/37,971 (23.7%) | 11.1/s | ETA: 44분 | Batch: 157.0s | 요청: 52,513 | 에러: 0 | 재시도: 67
⏱ 10,500/37,971 (27.7%) | 10.9/s | ETA: 42분 | Batch: 153.3s | 요청: 60,759 | 에러: 0 | 재시도: 81
⏱ 12,000/37,971 (31.6%) | 10.8/s | ETA: 40분 | Batch: 144.5s | 요청: 68,976 | 에러: 0 | 재시도: 92
⏱ 13,500/37,971 (35.6%) | 10.7/s | ETA: 38분 | Batch: 155.9s | 요청: 77,336 | 에러: 5 | 재시도: 121
⏱ 15,

In [24]:
# ==========================================================
# TMDB TV SERIES + SEASONS HYPER-OPTIMIZED ASYNC COLLECTOR
# 비동기 방식 + 병목 제거 + 캐싱
# ✨ 최종 개선: providers를 flatrate/rent/buy로 분리 + imdb_id 추가
# ==========================================================

import os
import asyncio
import aiohttp
from datetime import datetime, timedelta
from pathlib import Path
import pandas as pd
from dotenv import load_dotenv
from concurrent.futures import ThreadPoolExecutor
import time
from functools import lru_cache
import json

# ==========================================================
# Jupyter async 지원
# ==========================================================
try:
    import nest_asyncio
    nest_asyncio.apply()
    print("✅ nest_asyncio 적용 완료")
except:
    pass

# ==========================================================
# 설정
# ==========================================================
load_dotenv()
API_KEY = os.getenv("API_KEY")
if not API_KEY:
    raise ValueError("API_KEY missing")

BASE_URL = "https://api.themoviedb.org/3"
HEADERS = {"accept": "application/json"}

# 🔥 핵심: TMDB 실제 제한은 50/초이지만, 버스트를 고려해 40으로 안정화
MAX_CALLS_PER_SECOND = 25
TIMEOUT = aiohttp.ClientTimeout(total=15, connect=5, sock_read=10)
MAX_RETRIES = 2

START_DATE = "2013-01-01"
END_DATE = "2013-02-15"

SERIES_TEMP = Path("tv_series_temp.csv")
SEASONS_TEMP = Path("tv_seasons_temp.csv")

SERIES_CSV = "tv_series_2013_0101_0215_FULL.csv"
SEASONS_CSV = "tv_seasons_2013_0101_0215_FULL.csv"
SERIES_PARQ = "tv_series_2013_0101_0215_FULL.parquet"
SEASONS_PARQ = "tv_seasons_2013_0101_0215_FULL.parquet"

# 통계
stats = {"requests": 0, "errors": 0, "retries": 0, "start_time": None}

# ==========================================================
# 고성능 Rate Limiter (Lock-free 버전)
# ==========================================================
class LockFreeRateLimiter:
    """Lock 없이 동작하는 초고속 Rate Limiter"""
    def __init__(self, rate):
        self.rate = rate
        self.tokens = rate
        self.updated_at = time.monotonic()
    
    async def acquire(self):
        now = time.monotonic()
        elapsed = now - self.updated_at
        
        # 토큰 보충 (Lock 없이)
        self.tokens = min(self.rate, self.tokens + elapsed * self.rate)
        self.updated_at = now
        
        # 토큰 부족시 최소 대기
        if self.tokens < 1:
            sleep_time = (1 - self.tokens) / self.rate
            await asyncio.sleep(sleep_time)
            self.tokens = 1
        
        self.tokens -= 1

rate_limiter = LockFreeRateLimiter(MAX_CALLS_PER_SECOND)

# ==========================================================
# TMDB GET (재시도 로직 개선)
# ==========================================================
async def tmdb_get(session, path, params=None, retry=0):
    if retry >= MAX_RETRIES:
        stats["errors"] += 1
        return None

    params = params or {}
    params.setdefault("api_key", API_KEY)
    params.setdefault("language", "en-US")

    url = f"{BASE_URL}{path}"
    
    await rate_limiter.acquire()
    stats["requests"] += 1

    try:
        async with session.get(url, params=params, headers=HEADERS) as resp:
            # 429: Rate limit
            if resp.status == 429:
                stats["retries"] += 1
                retry_after = int(resp.headers.get("Retry-After", 2))
                await asyncio.sleep(retry_after)
                return await tmdb_get(session, path, params, retry + 1)
            
            # 404: Not found (정상 케이스)
            if resp.status == 404:
                return None
            
            # 5xx: 서버 에러
            if 500 <= resp.status < 600:
                stats["retries"] += 1
                await asyncio.sleep(0.5 ** retry)  # 지수 감소
                return await tmdb_get(session, path, params, retry + 1)
            
            resp.raise_for_status()
            return await resp.json()
    
    except asyncio.TimeoutError:
        stats["errors"] += 1
        if retry < MAX_RETRIES - 1:
            await asyncio.sleep(0.2)
            return await tmdb_get(session, path, params, retry + 1)
        return None
    
    except Exception as e:
        stats["errors"] += 1
        if retry < MAX_RETRIES - 1:
            await asyncio.sleep(0.1)
            return await tmdb_get(session, path, params, retry + 1)
        return None

# ==========================================================
# Discover
# ==========================================================
async def fetch_discover_page(session, page, gte, lte):
    params = {
        "sort_by": "popularity.desc",
        "first_air_date.gte": gte,
        "first_air_date.lte": lte,
        "page": page,
        "include_adult": "true",
    }
    data = await tmdb_get(session, "/discover/tv", params)
    if not data:
        return [], 1, 0
    return data.get("results", []), data.get("total_pages", 1), data.get("total_results", 0)

# ==========================================================
# ID 수집 (개선)
# ==========================================================
async def collect_ids_in_range(session, start, end, depth=0):
    res1, total_pages, total_results = await fetch_discover_page(session, 1, start, end)

    if total_results == 0:
        return set()

    if total_pages <= 500:
        ids = {r["id"] for r in res1}
        
        if total_pages > 1:
            # 🔥 개선: 청크로 나눠서 메모리 효율 개선
            chunk_size = 100
            for chunk_start in range(2, total_pages + 1, chunk_size):
                chunk_end = min(chunk_start + chunk_size, total_pages + 1)
                results = await asyncio.gather(
                    *[fetch_discover_page(session, p, start, end) for p in range(chunk_start, chunk_end)],
                    return_exceptions=True
                )
                for r in results:
                    if isinstance(r, tuple):
                        ids.update(x["id"] for x in r[0])
        
        return ids

    # 날짜 분할
    start_dt = datetime.strptime(start, "%Y-%m-%d")
    end_dt = datetime.strptime(end, "%Y-%m-%d")
    mid_dt = start_dt + (end_dt - start_dt) / 2
    mid = mid_dt.strftime("%Y-%m-%d")
    right_start = (mid_dt + timedelta(days=1)).strftime("%Y-%m-%d")

    left, right = await asyncio.gather(
        collect_ids_in_range(session, start, mid, depth + 1),
        collect_ids_in_range(session, right_start, end, depth + 1)
    )
    return left | right

# ==========================================================
# Helpers (캐싱 추가)
# ==========================================================
@lru_cache(maxsize=1024)
def list_to_str_cached(lst_tuple, key="name"):
    """리스트를 문자열로 변환 (캐싱)"""
    if not lst_tuple:
        return ""
    return ", ".join(str(i.get(key, "")) for i in lst_tuple if i.get(key))

def list_to_str(lst, key="name"):
    if not lst:
        return ""
    # 딕셔너리 리스트를 튜플로 변환하여 캐싱 가능하게
    try:
        lst_tuple = tuple(tuple(d.items()) if isinstance(d, dict) else d for d in lst)
        return list_to_str_cached(lst_tuple, key)
    except:
        return ", ".join(str(i.get(key, "")) for i in lst if i.get(key))

def list_ids_to_str(lst, key="id"):
    if not lst:
        return ""
    return ", ".join(str(i.get(key)) for i in lst if i.get(key))

# ==========================================================
# ✨ NEW: External IDs (IMDB ID)
# ==========================================================
async def fetch_external_ids(session, sid):
    """시리즈의 external IDs를 가져와서 IMDB ID 반환"""
    data = await tmdb_get(session, f"/tv/{sid}/external_ids")
    if not data:
        return ""
    return data.get("imdb_id", "")

# ==========================================================
# ✨ UPDATED: Providers (flatrate/rent/buy 분리)
# ==========================================================
async def fetch_providers(session, sid):
    """
    모든 국가의 streaming providers 수집
    flatrate(구독형), rent(대여), buy(구매) 타입별로 분리
    반환: (flatrate_dict, rent_dict, buy_dict)
    """
    data = await tmdb_get(session, f"/tv/{sid}/watch/providers")
    if not data:
        return "{}", "{}", "{}"
    
    providers_results = data.get("results", {})
    
    # 국가별 OTT 정보 수집
    providers_flatrate = {}
    providers_rent = {}
    providers_buy = {}
    
    for country, info in providers_results.items():
        # flatrate: 구독형 (Netflix, Disney+ 등)
        flatrate = info.get("flatrate", [])
        if flatrate:
            provider_names = [p.get("provider_name") for p in flatrate if p.get("provider_name")]
            if provider_names:
                providers_flatrate[country] = provider_names
        
        # rent: 대여
        rent = info.get("rent", [])
        if rent:
            provider_names = [p.get("provider_name") for p in rent if p.get("provider_name")]
            if provider_names:
                providers_rent[country] = provider_names
        
        # buy: 구매
        buy = info.get("buy", [])
        if buy:
            provider_names = [p.get("provider_name") for p in buy if p.get("provider_name")]
            if provider_names:
                providers_buy[country] = provider_names
    
    # JSON 문자열로 변환하여 반환
    return (
        json.dumps(providers_flatrate, ensure_ascii=False) if providers_flatrate else "{}",
        json.dumps(providers_rent, ensure_ascii=False) if providers_rent else "{}",
        json.dumps(providers_buy, ensure_ascii=False) if providers_buy else "{}"
    )

# ==========================================================
# Single Season
# ==========================================================
async def fetch_single_season(session, sid, sname, net_names, net_ids, meta):
    sn = meta.get("season_number")
    if sn is None:
        return None

    data = await tmdb_get(session, f"/tv/{sid}/season/{sn}")

    if not data:
        return {
            "_id": f"{sid}_{sn}",
            "season_id": meta.get("id"),
            "series_id": sid,
            "series_name": sname,
            "season_number": sn,
            "name": meta.get("name"),
            "air_date": meta.get("air_date"),
            "overview": meta.get("overview"),
            "vote_average": meta.get("vote_average"),
            "vote_count": meta.get("vote_count"),
            "network_names": net_names,
            "network_ids": net_ids,
            "total_episodes": meta.get("episode_count"),
            "avg_episode_runtime": None,
            "poster_path": meta.get("poster_path"),
        }

    eps = data.get("episodes") or []
    runtimes = [ep["runtime"] for ep in eps if ep.get("runtime")]
    avg_rt = sum(runtimes) / len(runtimes) if runtimes else None

    return {
        "_id": data.get("_id") or f"{sid}_{sn}",
        "season_id": data.get("id"),
        "series_id": sid,
        "series_name": sname,
        "season_number": data.get("season_number"),
        "name": data.get("name"),
        "air_date": data.get("air_date"),
        "overview": data.get("overview"),
        "vote_average": data.get("vote_average"),
        "vote_count": data.get("vote_count"),
        "network_names": net_names,
        "network_ids": net_ids,
        "total_episodes": len(eps),
        "avg_episode_runtime": avg_rt,
        "poster_path": data.get("poster_path"),
    }

# ==========================================================
# ✨ UPDATED: TV Details + Seasons + IMDB ID + Providers
# ==========================================================
async def fetch_tv_details_and_seasons(session, sid):
    params = {"append_to_response": "reviews,keywords,aggregate_credits"}
    data = await tmdb_get(session, f"/tv/{sid}", params)
    if not data:
        return None, []

    genres = data.get("genres") or []
    networks = data.get("networks") or []
    last_ep = data.get("last_episode_to_air") or {}

    net_names = list_to_str(networks)
    net_ids = list_ids_to_str(networks)

    series = {
        "id": data.get("id"),
        "title": data.get("name"),
        "type": "tv_series",
        "adult": data.get("adult"),
        "backdrop_path": data.get("backdrop_path"),
        "created_by": list_to_str(data.get("created_by")),
        "episode_run_time": ", ".join(map(str, data.get("episode_run_time", []))) or "",
        "first_air_date": data.get("first_air_date"),
        "genres": list_to_str(genres),
        "genre_ids": ", ".join(str(g["id"]) for g in genres if g.get("id")),
        "homepage": data.get("homepage"),
        "in_production": data.get("in_production"),
        "languages": ", ".join(data.get("languages", [])) or "",
        "last_air_date": data.get("last_air_date"),
        "last_episode_to_air_id": last_ep.get("id"),
        "last_episode_to_air_name": last_ep.get("name"),
        "last_episode_to_air_overview": last_ep.get("overview"),
        "last_episode_to_air_vote_average": last_ep.get("vote_average"),
        "last_episode_to_air_vote_count": last_ep.get("vote_count"),
        "last_episode_to_air_air_date": last_ep.get("air_date"),
        "last_episode_to_air_episode_number": last_ep.get("episode_number"),
        "last_episode_to_air_production_code": last_ep.get("production_code"),
        "last_episode_to_air_runtime": last_ep.get("runtime"),
        "last_episode_to_air_season_number": last_ep.get("season_number"),
        "last_episode_to_air_show_id": last_ep.get("show_id"),
        "last_episode_to_air_still_path": last_ep.get("still_path"),
        "next_episode_to_air": str(data.get("next_episode_to_air")) if data.get("next_episode_to_air") else "",
        "networks": net_names,
        "number_of_episodes": data.get("number_of_episodes"),
        "number_of_seasons": data.get("number_of_seasons"),
        "origin_country": ", ".join(data.get("origin_country", [])) or "",
        "original_language": data.get("original_language"),
        "original_name": data.get("original_name"),
        "overview": data.get("overview"),
        "popularity": data.get("popularity"),
        "poster_path": data.get("poster_path"),
        "production_companies": list_to_str(data.get("production_companies")),
        "production_countries": list_to_str(data.get("production_countries")),
        "seasons": "; ".join(
            f"S{s['season_number']}: {s.get('name')} ({s.get('episode_count')} eps)"
            for s in (data.get("seasons") or []) if s.get("season_number") is not None
        ),
        "spoken_languages": list_to_str(data.get("spoken_languages")),
        "status": data.get("status"),
        "tagline": data.get("tagline"),
        "type_detail": data.get("type"),
        "vote_average": data.get("vote_average"),
        "vote_count": data.get("vote_count"),
    }

    # 리뷰
    rv_items = data.get("reviews", {}).get("results", [])
    series["review"] = " || ".join([
        f"{r.get('author', '')}({r.get('author_details', {}).get('rating', '')}): {(r.get('content') or '').replace(chr(10), ' ')[:200]}"
        for r in rv_items[:5]
    ])

    # 키워드
    kw_items = data.get("keywords", {}).get("results", [])
    series["keyword"] = ", ".join([k["name"] for k in kw_items[:20] if k.get("name")])

    # Cast & Crew
    credits = data.get("aggregate_credits", {})
    cast = credits.get("cast") or []
    crew = credits.get("crew") or []

    series["top_cast"] = ", ".join([c["name"] for c in cast[:4] if c.get("name")])

    dirs, wrs = set(), set()
    for c in crew:
        nm = c.get("name")
        if nm:
            for job in (c.get("jobs") or []):
                jn = job.get("job", "")
                if "Director" in jn:
                    dirs.add(nm)
                if jn in ["Writer", "Screenplay", "Story"]:
                    wrs.add(nm)

    series["directors"] = ", ".join(sorted(dirs)[:10])
    series["writers"] = ", ".join(sorted(wrs)[:10])

    # 🔥 providers + external_ids + seasons 병렬 수집
    seasons_meta = [s for s in (data.get("seasons") or []) if s.get("season_number") is not None]
    
    tasks = [
        fetch_providers(session, sid),      # 0: (flatrate, rent, buy)
        fetch_external_ids(session, sid),   # 1: imdb_id
    ]
    tasks.extend([
        fetch_single_season(session, sid, data.get("name", ""), net_names, net_ids, s)
        for s in seasons_meta
    ])
    
    results = await asyncio.gather(*tasks, return_exceptions=True)
    
    # Providers 처리 (flatrate, rent, buy)
    if isinstance(results[0], tuple) and len(results[0]) == 3:
        series["providers_flatrate"] = results[0][0]
        series["providers_rent"] = results[0][1]
        series["providers_buy"] = results[0][2]
    else:
        series["providers_flatrate"] = "{}"
        series["providers_rent"] = "{}"
        series["providers_buy"] = "{}"
    
    # IMDB ID 처리
    series["imdb_id"] = results[1] if isinstance(results[1], str) else ""
    
    # Seasons 처리
    season_records = [r for r in results[2:] if r and not isinstance(r, Exception)]

    return series, season_records

# ==========================================================
# 비동기 CSV 쓰기
# ==========================================================
executor = ThreadPoolExecutor(max_workers=2)

async def save_to_csv_async(df, path, mode, header):
    loop = asyncio.get_event_loop()
    await loop.run_in_executor(
        executor,
        lambda: df.to_csv(path, mode=mode, header=header, index=False, encoding="utf-8-sig")
    )

# ==========================================================
# 결측치 보완
# ==========================================================
def fill_series_gaps(series_df, seasons_df):
    print("\n📊 시즌 기반 결측치 보완")

    rtmap = seasons_df.groupby("series_id")["avg_episode_runtime"].mean().round()
    mask = series_df["episode_run_time"].astype(str).str.strip().isin(["", "nan"])
    series_df.loc[mask, "episode_run_time"] = (
        series_df.loc[mask, "id"].map(rtmap).fillna(0).astype(int).astype(str)
    )

    epmap = seasons_df.groupby("series_id")["total_episodes"].sum()
    mask2 = series_df["number_of_episodes"].isna()
    series_df.loc[mask2, "number_of_episodes"] = series_df.loc[mask2, "id"].map(epmap)

    seasmap = seasons_df.groupby("series_id")["season_number"].nunique()
    mask3 = series_df["number_of_seasons"].isna()
    series_df.loc[mask3, "number_of_seasons"] = series_df.loc[mask3, "id"].map(seasmap)

    lastair = seasons_df.sort_values("air_date").groupby("series_id")["air_date"].last()
    ladmask = series_df["last_air_date"].astype(str).str.strip().isin(["", "nan", "NaT"])
    series_df.loc[ladmask, "last_air_date"] = series_df.loc[ladmask, "id"].map(lastair)

    print("✅ 완료\n")
    return series_df

# ==========================================================
# MAIN
# ==========================================================
async def main():
    print("=" * 90)
    print("🚀 TMDB HYPER-OPTIMIZED ASYNC COLLECTOR")
    print("✨ 최종 개선: providers(flatrate/rent/buy 분리) + IMDB ID 추가")
    print("=" * 90)
    stats["start_time"] = datetime.now()
    t0 = datetime.now()

    for f in [SERIES_TEMP, SEASONS_TEMP]:
        if f.exists():
            f.unlink()

    # 🔥 최적 설정
    connector = aiohttp.TCPConnector(
        limit=150,
        limit_per_host=75,
        ttl_dns_cache=600,
        force_close=False,
        enable_cleanup_closed=True
    )

    async with aiohttp.ClientSession(connector=connector, timeout=TIMEOUT) as session:
        # 1) ID 수집
        print("\n📌 1단계: ID 수집")
        ids = sorted(list(await collect_ids_in_range(session, START_DATE, END_DATE)))
        print(f"✨ 총 ID: {len(ids):,}개")

        # 2) 상세 수집
        print("\n📌 2단계: 상세 + 시즌 + IMDB ID + Providers 수집")
        batch_size = 1500
        sem = asyncio.Semaphore(30)  # 🔥 안정적인 동시성

        async def fetch_one(i):
            async with sem:
                return await fetch_tv_details_and_seasons(session, i)

        processed = 0
        last_print = time.time()
        
        for start_idx in range(0, len(ids), batch_size):
            batch_start = time.time()
            part = ids[start_idx:start_idx+batch_size]
            
            results = await asyncio.gather(*[fetch_one(x) for x in part], return_exceptions=True)
            
            bs, bse = [], []
            for r in results:
                if isinstance(r, tuple) and r[0]:
                    s, ss = r
                    bs.append(s)
                    bse.extend(ss)
                processed += 1

            # CSV 쓰기
            if bs:
                await save_to_csv_async(pd.DataFrame(bs), SERIES_TEMP, "a", not SERIES_TEMP.exists())
            if bse:
                await save_to_csv_async(pd.DataFrame(bse), SEASONS_TEMP, "a", not SEASONS_TEMP.exists())

            # 진행 상황 (5초마다)
            now = time.time()
            if now - last_print >= 5:
                elapsed = (datetime.now() - t0).total_seconds()
                rate = processed / elapsed if elapsed > 0 else 0
                eta = (len(ids) - processed) / rate / 60 if rate > 0 else 0
                batch_time = now - batch_start
                print(f"⏱ {processed:,}/{len(ids):,} ({processed/len(ids)*100:.1f}%) | "
                      f"{rate:.1f}/s | ETA: {eta:.0f}분 | Batch: {batch_time:.1f}s | "
                      f"요청: {stats['requests']:,} | 에러: {stats['errors']} | 재시도: {stats['retries']}")
                last_print = now

    # 3) 최종 처리
    print("\n📌 3단계: 최종 처리")
    df_series = pd.read_csv(SERIES_TEMP)
    df_seasons = pd.read_csv(SEASONS_TEMP) if SEASONS_TEMP.exists() else pd.DataFrame()

    SERIES_COLS = [
        "id","title","type","adult","backdrop_path","created_by","episode_run_time",
        "first_air_date","genres","genre_ids","homepage","in_production","languages",
        "last_air_date","last_episode_to_air_id","last_episode_to_air_name",
        "last_episode_to_air_overview","last_episode_to_air_vote_average",
        "last_episode_to_air_vote_count","last_episode_to_air_air_date",
        "last_episode_to_air_episode_number","last_episode_to_air_production_code",
        "last_episode_to_air_runtime","last_episode_to_air_season_number",
        "last_episode_to_air_show_id","last_episode_to_air_still_path",
        "next_episode_to_air","networks","number_of_episodes","number_of_seasons",
        "origin_country","original_language","original_name","overview","popularity",
        "poster_path","production_companies","production_countries","seasons",
        "spoken_languages","status","tagline","type_detail","vote_average",
        "vote_count","review","keyword","top_cast","directors","writers",
        "providers_flatrate","providers_rent","providers_buy","imdb_id"  # ✨ 변경된 컬럼들
    ]

    SEASON_COLS = [
        "_id","season_id","series_id","series_name","season_number","name","air_date",
        "overview","vote_average","vote_count","network_names","network_ids",
        "total_episodes","avg_episode_runtime","poster_path"
    ]

    df_series = df_series[SERIES_COLS].drop_duplicates(subset=["id"])
    if not df_seasons.empty:
        df_seasons = df_seasons[SEASON_COLS].drop_duplicates(subset=["series_id","season_number"])
        df_series = fill_series_gaps(df_series, df_seasons)

    df_series.to_csv(SERIES_CSV, index=False, encoding="utf-8-sig")
    df_seasons.to_csv(SEASONS_CSV, index=False, encoding="utf-8-sig")

    try:
        df_series.to_parquet(SERIES_PARQ, index=False)
        df_seasons.to_parquet(SEASONS_PARQ, index=False)
    except:
        pass

    for f in [SERIES_TEMP, SEASONS_TEMP]:
        if f.exists():
            f.unlink()

    elapsed = (datetime.now() - t0).total_seconds()/60

    print("="*90)
    print("🎉 수집 완료!")
    print("="*90)
    print(f"📌 Series: {len(df_series):,}개 | Seasons: {len(df_seasons):,}개")
    print(f"📌 API 요청: {stats['requests']:,}회 | 에러: {stats['errors']} | 재시도: {stats['retries']}")
    print(f"📌 평균 속도: {len(df_series)/elapsed:.1f} series/분")
    print(f"⏱ 총 시간: {elapsed:.1f}분 ({elapsed/60:.1f}시간)")
    print("="*90)
    
    # 샘플 데이터 출력
    print("\n📊 샘플 Providers 데이터:")
    if not df_series.empty and 'providers_flatrate' in df_series.columns:
        sample = df_series[df_series['providers_flatrate'] != '{}'].head(1)
        if not sample.empty:
            print(f"Title: {sample.iloc[0]['title']}")
            print(f"Flatrate: {sample.iloc[0]['providers_flatrate'][:200]}...")
            print(f"IMDB ID: {sample.iloc[0]['imdb_id']}")
    
    executor.shutdown(wait=False)

if __name__ == "__main__":
    asyncio.run(main())

✅ nest_asyncio 적용 완료
🚀 TMDB HYPER-OPTIMIZED ASYNC COLLECTOR
✨ 최종 개선: providers(flatrate/rent/buy 분리) + IMDB ID 추가

📌 1단계: ID 수집
✨ 총 ID: 768개

📌 2단계: 상세 + 시즌 + IMDB ID + Providers 수집
⏱ 768/768 (100.0%) | 10.9/s | ETA: 0분 | Batch: 68.9s | 요청: 3,928 | 에러: 0 | 재시도: 8

📌 3단계: 최종 처리

📊 시즌 기반 결측치 보완
✅ 완료

🎉 수집 완료!
📌 Series: 768개 | Seasons: 1,577개
📌 API 요청: 3,928회 | 에러: 0 | 재시도: 8
📌 평균 속도: 648.6 series/분
⏱ 총 시간: 1.2분 (0.0시간)

📊 샘플 Providers 데이터:
Title: House of Cards
Flatrate: {"AD": ["Netflix"], "AE": ["Netflix"], "AG": ["Netflix"], "AL": ["Netflix"], "AO": ["Netflix"], "AR": ["Netflix"], "AT": ["Netflix"], "AU": ["Netflix"], "AZ": ["Netflix"], "BA": ["Netflix"], "BB": ["N...
IMDB ID: tt1856010


In [1]:
# 1. 깨진 줄을 무시하고 정상 줄만 모아서 clean 파일 생성
def clean_csv(input_path, output_path):
    with open(input_path, "r", errors="ignore") as infile, open(output_path, "w", encoding="utf-8") as outfile:
        for line in infile:
            if line.count(",") >= 5:  # 최소 5개 이상 쉼표 있으면 정상 라인으로 간주
                outfile.write(line)

clean_csv("tv_series_2005_2015_FULL.csv", "tv_series_clean.csv")
clean_csv("tv_seasons_2005_2015_FULL.csv", "tv_seasons_clean.csv")


In [3]:
import pandas as pd
df_series = pd.read_csv("tv_series_clean.csv", engine="python")
df_seasons = pd.read_csv("tv_seasons_clean.csv", engine="python")


In [4]:
def get_null_ratio(df):
    null_ratio = (df.isnull().sum() / len(df)) * 100
    return null_ratio.sort_values(ascending=False)

series_null_ratio = get_null_ratio(df_series)
seasons_null_ratio = get_null_ratio(df_seasons)

print("\n===== TV SERIES NULL RATIO (%) =====")
print(series_null_ratio)

print("\n===== TV SEASONS NULL RATIO (%) =====")
print(seasons_null_ratio)



===== TV SERIES NULL RATIO (%) =====
next_episode_to_air                    99.586516
last_episode_to_air_production_code    99.415328
review                                 98.638399
tagline                                96.597314
last_episode_to_air_still_path         80.208059
writers                                78.182776
created_by                             74.601001
last_episode_to_air_overview           69.109824
keyword                                67.761391
homepage                               67.121412
directors                              67.058204
production_companies                   58.401370
imdb_id                                48.206479
backdrop_path                          47.102976
last_episode_to_air_runtime            46.233869
production_countries                   43.745062
overview                               43.305241
networks                               37.263629
top_cast                               30.990255
genre_ids                      

In [15]:
import pandas as pd

# 데이터 불러오기
df_series = pd.read_csv("tv_series_2005_2015_FULL.csv")
df_seasons = pd.read_csv("tv_seasons_2005_2015_FULL.csv")

# 결측률 계산 함수
def missing_rate(df):
    return df.isnull().mean() * 100   # 퍼센트로 표시

# 각 파일의 컬럼별 결측률
series_missing = missing_rate(df_series)
seasons_missing = missing_rate(df_seasons)

print("=== tv_series_clean.csv 결측률 ===")
print(series_missing)

print("\n=== tv_seasons_clean.csv 결측률 ===")
print(seasons_missing)


C:\Users\Public\Documents\ESTsoft\CreatorTemp\ipykernel_11924\4136345038.py:4: DtypeWarning: Columns (3,21) have mixed types. Specify dtype option on import or set low_memory=False.
  df_series = pd.read_csv("tv_series_2005_2015_FULL.csv")


=== tv_series_clean.csv 결측률 ===
id                                      0.000000
title                                   0.002634
type                                    0.000000
adult                                   0.002634
backdrop_path                          47.102976
created_by                             74.601001
episode_run_time                        0.000000
first_air_date                          0.002634
genres                                 29.007111
genre_ids                              29.007111
homepage                               67.121412
in_production                           0.000000
languages                              26.099552
last_air_date                           0.079010
last_episode_to_air_id                  0.144851
last_episode_to_air_name                0.150119
last_episode_to_air_overview           69.109824
last_episode_to_air_vote_average        0.144851
last_episode_to_air_vote_count          0.144851
last_episode_to_air_air_date         

In [16]:
df_series.shape

(37970, 54)

In [19]:
# 방법 1: shape[0] 사용
count = df_series[(df_series['vote_count'] >= 10) & (df_series['imdb_id'].notna())].shape[0]
print(count)


3851
